In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import json
import numpy as np
import pandas as pd
import scipy as sp
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import matplotlib.ticker as ticker
from PIL import Image

from experiment_storage.file_paths import FilePaths
from experiment_storage.storage import EvaluationStorage

vrp_variants = ["TSP", "CVRP", "HVRP", "VRPTW", "LRP"]
solomon_types = ["O", "C", "R"]
# vrp_variants = ["LRP"]
# solomon_types = ["O"]


manual_legend = {'dqn': 'GRLOS',
                'dqnprob': 'GRLOS-M',
                'vrwmdp': 'LRW',
                'hybrid': 'Hybrid',
                'classicrw': "CRW",
                'randommdp': "RAN",
                'random': "RAN",
                'dqnbaseline': 'RLB',
                'dqnbaselineprob': 'RLB-M',
                }

newname_palette_v0 = {'GRLOS':    (0, 128, 255),   # blue
                    'GRLOS-M': (163, 140, 244), # purple 
                    'LRW':     (21, 220, 74),   # green
                    'Hybrid':  (244, 97, 221),  # pink 
                    'CRW':     (165, 88, 11),  # brown
                    'RAN':     (252, 141, 98),  # orange
                    'RLB':     (255, 51, 51),   # red
                    'RLB-M':   (150, 163, 49),  # olive 
                    }
oldname_palette_v0 = {'dqn':          (0, 128, 255),   # blue
                    'dqnprob':        (163, 140, 244), # purple 
                    'vrwmdp':         (21, 220, 74),   # green
                    'hybrid':         (244, 97, 221),  # pink 
                    'classicrw':      (165, 88, 11),   # brown
                    'randommdp':      (252, 141, 98),  # orange
                    'random':         (252, 141, 98),  # orange
                    'dqnbaseline':    (255, 51, 51),   # red
                    'dqnbaselineprob':(150, 163, 49),  # olive 
                    }
oldname_palette_v1 = {'dqn':    (0, 114, 178),  # blue
                    'dqnprob': (152, 78,  163), # purple 
                    'vrwmdp':     (0, 158, 115),  # green
                    'hybrid':    (247, 129, 191),  # pink 
                    'classicrw':     (166, 86,  40),  # brown
                    'randommdp':     (255, 127, 0),  # orange
                    'random':     (255, 127, 0),  # orange
                    'dqnbaseline':     (228, 26,  28),   # red
                    'dqnbaselineprob':   (222, 222, 0),  # yellow 
                    }

new_palette = {key: tuple(value / 255 for value in values) for key, values in newname_palette_v0.items()}
old_palette = {key: tuple(value / 255 for value in values) for key, values in oldname_palette_v1.items()}



parent_dir = '/Users/shunee/github/All-Experiment-Data/' 

exp_1_file_name_1 = 'all_variants_all_instances_proportional_tours_multiple_alns_steps'
exp_1_file_name_2 = 'rc_all_variants_redone'

exp_2_file_name = 'all_variants_all_instances_proportional_tours_multiple_alns_steps' 

# exp_3_file_name = 'gehring_homberger_experiments_3'
# exp_3_file_name = 'gehring_homberger_experiments_complete'
exp_3_file_name = 'gehring_homberger_experiments_withDQN'

exp_4_file_name = 'all_variants_timingresults'

exp_5_file_name = 'all_variants_all_instances_proportional_tours_multiple_alns_steps' 

exp_6_file_name_1 = 'all_variants_baseline'
exp_6_file_name_2 = 'all_variants_all_instances_proportional_tours_multiple_alns_steps'
exp_6_file_name_3 = 'rc_all_variants_redone'

exp_7_file_name = 'all_variants_proportional_tours'


---
## 1. MDP Evaluation 
- Train the model up to 50 nodes and extrapolate to up to 100 nodes
- Fixed evaluation budget = 20

### 1.1 barplot + 1.2 barplot with ci
averaged from 128 seeds using pivot table during data extraction 

In [ ]:
print_out_txt = False

def compute_ci(data, confidence=0.95):
    a = np.array(data)
    n = len(a)
    se = sp.stats.sem(a)
    h = se * sp.stats.t.ppf((1 + confidence) / 2., n-1)
    return h

def mdp_barplot(evaluation_type, folder_name, include_ci):

    for solomon_type in solomon_types: 
        for vrp_variant in vrp_variants: 
            
            #--------------------------------
            # read in corresponding data: 
            #--------------------------------
            if solomon_type != "O":
                exp_id = f'{solomon_type}101200_diffinitrand{vrp_variant}'
                fp_in = FilePaths(parent_dir + exp_1_file_name_1, exp_id, setup_directories=False)
            else:
                exp_id = f'{solomon_type}101200_diffinitrandredone{vrp_variant}'
                fp_in = FilePaths(parent_dir + exp_1_file_name_2, exp_id, setup_directories=False)
                
            storage = EvaluationStorage(fp_in)
            results = storage.get_evaluation_data(exp_id)
            results = [r for r in results if r["is_best_hyps"]]
            rows_df = pd.DataFrame(results)
            
            results_table = pd.pivot_table(rows_df, values='cummulative_reward', columns=["algorithm"], index=["cust_number"]).reset_index()
        
            #--------------------------------
            # include confidence interval: 
            #--------------------------------
            format_ci_dict = {}
            exp_details = storage.get_experiment_details(exp_id)
            for agent_name in exp_details["agents"]:
                if agent_name.startswith("dqnprob") or agent_name.startswith("classicrw"):
                    continue

                cis_data = []
                for cust_number in exp_details["experiment_conditions"]["num_customers_test"]:
                    data = rows_df[(rows_df["algorithm"] == agent_name) & (rows_df["cust_number"] == cust_number)]["cummulative_reward"]

                    cis = compute_ci(data)
                    cis_data.append(cis)

                srs = pd.Series(cis_data)
                results_table[agent_name + "_ci"] = srs
                results_table["win_dqn"] = results_table["dqn"] > results_table["vrwmdp"]
                results_table.sort_index(axis=1, inplace=True)

            pd.set_option('display.max_columns', None)
            pd.set_option('display.width', 1000)

            #--------------------------------
            # print out dataframe as txt: 
            #--------------------------------
            if print_out_txt:
                with open("mdp_evaluation.txt", "a") as file:
                    file.write("=" * 20 + '\n')
                    file.write(exp_id + '\n')
                    file.write("=" * 20 + '\n')
                    results_table_string = results_table.to_string(index=False)
                    file.write(results_table_string + '\n')
                    file.write('\n')

            #--------------------------------
            # plot out bar plot/histogram:   
            #--------------------------------
            title_name = f'MDP Training ({vrp_variant}, {solomon_type})'
            x_axis_column = "cust_number"
            x_label = 'Instance Size'
            y_label = 'Cum. Rewards'
            columns_to_plot = ['cust_number', 'dqn', 'vrwmdp', 'randommdp']

            original_labels = [key for key, value in manual_legend.items() if key in rows_df.algorithm.unique()]
            new_labels = [value for key, value in manual_legend.items() if key in rows_df.algorithm.unique()]
            
            df = pd.DataFrame(results_table)
            df_melted = df.melt(id_vars=x_axis_column, value_vars=columns_to_plot, var_name='method', value_name='value')  # Melt the DataFrame to convert it to the long format

            sns.set(style='darkgrid')
            plt.figure(figsize=(10, 6))
            ax = plt.subplot(111)
            sns.barplot(x=x_axis_column, y='value', hue='method', data=df_melted, ci=None, ax=ax, hue_order=original_labels)


            #--------------------------------
            # include ci or not? 
            #--------------------------------
            if include_ci: 
                
                ## Calculate the position for the error bars. Add error bars to the plot
                bar_width = 0.27
                number_of_groups = 9
                ci_types = [name+"_ci" for name in columns_to_plot if name != 'cust_number']

                for ci_type in ci_types: 
                    if ci_type == "dqn_ci": 
                        x_positions = [x_position - bar_width for x_position in range(number_of_groups)]
                        y_values = results_table['dqn']
                    elif ci_type == "vrwmdp_ci": 
                        x_positions = [x_position for x_position in range(number_of_groups)]
                        y_values = results_table['vrwmdp']
                    elif ci_type == "randommdp_ci": 
                        x_positions = [x_position + bar_width for x_position in range(number_of_groups)]
                        y_values = results_table['randommdp']

                    ci_values = results_table[ci_type]
                    ax.errorbar(x=x_positions, y=y_values, yerr=[ci_values, ci_values], fmt='.', ecolor='black')
                    

            plt.subplots_adjust(top=0.9, left=0.1)
            plt.title(title_name, fontsize=16)
            plt.xlabel(x_label, fontsize=16)
            plt.ylabel(y_label, fontsize=16)
            plt.yticks(fontsize=14)
            # plt.ylim([0, 2500])
        
            new_x_labels = [10*x for x in range(2,11)]   
            plt.xticks(range(len(new_x_labels)), new_x_labels, fontsize=16)
            # plt.xticks(fontsize=16)

            #--------------------------------
            # Customize the legend. Replace the labels inside the legend
            #--------------------------------
            legend = plt.legend(borderaxespad=0., fontsize=14) #loc='upper left', bbox_to_anchor=(0.06, 0.9), bbox_to_anchor=(1, 1)
            for text, label in zip(legend.get_texts(), new_labels):
                text.set_text(label)

            #--------------------------------
            # print out plots in given folder: 
            #--------------------------------
            if not os.path.exists(folder_name):
                os.mkdir(folder_name)

            plt.tight_layout()
            plt.savefig(os.path.join(folder_name, f'{evaluation_type}_{vrp_variant}_solo{solomon_type}'), dpi=300)
            # plt.show()
            plt.close()

In [ ]:
evaluation_type = 'mdp-barplot'
folder_name = "1.1-" + evaluation_type
mdp_barplot(evaluation_type, folder_name, include_ci=False)

evaluation_type = 'mdp-barplot-with-ci'
folder_name = "1.2-" + evaluation_type
mdp_barplot(evaluation_type, folder_name, include_ci=True)

### 1.3 barplot (boxplot) - individual nodes
keep all the data points without aggregating, plot out boxplot using all data

In [ ]:
print_csv_file=False

def mdp_boxplot(evaluation_type, folder_name): 

    for solomon_type in solomon_types: 
        for vrp_variant in vrp_variants: 
        
            #--------------------------------
            # read in corresponding data: 
            #--------------------------------
            if solomon_type != "O":
                exp_id = f'{solomon_type}101200_diffinitrand{vrp_variant}'
                fp_in = FilePaths(parent_dir + exp_1_file_name_1, exp_id, setup_directories=False)
            else:
                exp_id = f'{solomon_type}101200_diffinitrandredone{vrp_variant}'
                fp_in = FilePaths(parent_dir + exp_1_file_name_2, exp_id, setup_directories=False)
                
            storage = EvaluationStorage(fp_in)
            results = storage.get_evaluation_data(exp_id, task_type="eval") 
            results = [r for r in results if r["is_best_hyps"]]
            rows_df = pd.DataFrame(results)

            if print_csv_file == True:
                rows_df.to_csv("df_table_mdp_" + exp_id + ".csv", sep=',', index=False)


            #--------------------------------
            # plot out box plot:   
            #--------------------------------
            title_name = f'MDP Training ({vrp_variant}, {solomon_type})'
            x_value = "cust_number"
            x_label = 'Instance Size'
            y_value = 'cummulative_reward'
            y_label = 'Cum. Rewards'
            category = "algorithm"            
            
            original_labels = [key for key, value in manual_legend.items() if key in rows_df.algorithm.unique()]
            new_labels = [value for key, value in manual_legend.items() if key in rows_df.algorithm.unique()]
            
            sns.set(style='darkgrid')
            plt.figure(figsize=(10, 6))
            ax = plt.subplot(111)
            sns.boxplot(x=x_value, y=y_value, hue=category, data=rows_df, ax=ax, hue_order=original_labels)

            plt.subplots_adjust(top=0.9, left=0.1)
            plt.title(title_name, fontsize=16)
            plt.xlabel(x_label, fontsize=14)
            plt.ylabel(y_label, fontsize=14)

            #--------------------------------
            # Customize the legend. Replace the labels inside the legend
            #--------------------------------
            legend = plt.legend(borderaxespad=0., fontsize=14) #loc='upper left', bbox_to_anchor=(0.06, 0.9), bbox_to_anchor=(1, 1)
            for text, label in zip(legend.get_texts(), new_labels):
                text.set_text(label)
            
            if not os.path.exists(folder_name):
                os.mkdir(folder_name)

            plt.tight_layout()
            plt.savefig(os.path.join(folder_name, f'{evaluation_type}_{vrp_variant}_solo{solomon_type}.png'), dpi=300)
            plt.show()


In [ ]:
evaluation_type = 'mdp-barplot'
folder_name = "1.3-" + evaluation_type
mdp_boxplot(evaluation_type, folder_name)

### Summary Plot (1.2 barplot with ci)

In [ ]:
def mdp_barplot_summary_plot(solomon_types=solomon_types): 

    sns.set(style='darkgrid')
    fig, ax = plt.subplots(figsize=(20,20), nrows=5, ncols=3)

    column_position = -1
    
    for solomon_type in solomon_types: 
        row_position = -1
        column_position += 1
        
        for vrp_variant in vrp_variants: 
            row_position += 1

            #--------------------------------
            # read in corresponding data: 
            #--------------------------------
            if solomon_type != "O":
                exp_id = f'{solomon_type}101200_diffinitrand{vrp_variant}'
                fp_in = FilePaths(parent_dir + exp_1_file_name_1, exp_id, setup_directories=False)
            else:
                exp_id = f'{solomon_type}101200_diffinitrandredone{vrp_variant}'
                fp_in = FilePaths(parent_dir + exp_1_file_name_2, exp_id, setup_directories=False)
                
            storage = EvaluationStorage(fp_in)
            results = storage.get_evaluation_data(exp_id, task_type="eval") 
            results = [r for r in results if r["is_best_hyps"]]
            rows_df = pd.DataFrame(results)
            
            #--------------------------------
            # plot out box plot:   
            #--------------------------------
            title_name = f'MDP Evaluation ({vrp_variant}, {solomon_type})'
            x_value = "cust_number"
            x_label = 'Instance Size'
            y_value = 'cummulative_reward'
            y_label = 'Cummulative Rewards'
            category = "algorithm"            
            
            original_labels = [key for key, value in manual_legend.items() if key in rows_df.algorithm.unique()]
            new_labels = [value for key, value in manual_legend.items() if key in rows_df.algorithm.unique()]
            
            # Create a color palette with a consistent order for each unique algorithm
            # palette = {alg: sns.color_palette('husl', n_colors=len(new_labels))[i] for i, alg in enumerate(original_labels)}
            specific_palette = {old_name: colour for old_name, colour in old_palette.items() if old_name in original_labels}

            sns.barplot(x=x_value, y=y_value, hue=category, data=rows_df, ax=ax[row_position, column_position], 
                        hue_order=original_labels, palette=specific_palette, ci=95, errcolor='red')


            subplot_solomon_type = solomon_type if solomon_type!= 'O' else 'RC'
            ax[row_position, column_position].set_title(f'{vrp_variant}, (type-{subplot_solomon_type})', fontsize=25)
            ax[row_position, column_position].set_xlabel('', fontsize=22)
            ax[row_position, column_position].set_ylabel('', fontsize=22)
            ax[row_position, column_position].tick_params(axis='x', labelsize=16)
            ax[row_position, column_position].tick_params(axis='y', labelsize=12)

            #--------------------------------
            # Customize the legend. Replace the labels inside the legend
            #--------------------------------
            legend = ax[row_position, column_position].get_legend()
            for text, label in zip(legend.get_texts(), new_labels):
                text.set_text(label)

    fig.supxlabel(x_label, fontsize=36)
    fig.supylabel(y_label, fontsize=36)

    fig.subplots_adjust(left=0.1, bottom=0.1)

    plt.tight_layout()
    plt.savefig(f"1_MDP_evaluation_barplot.pdf", 
                            bbox_inches='tight',
                            dpi=300)
    plt.close()


mdp_barplot_summary_plot(solomon_types=solomon_types)

---
## 2. ALNS Evaluation

### 2.1 lineplot without ci
averaged objective using pivot table when extracting data

In [ ]:
print_out_txt = False
filter_agent = None

def alns_lineplot(evaluation_type, folder_name, avg_or_min):

    for solomon_type in solomon_types: 
        for vrp_variant in vrp_variants: 
            
            #--------------------------------
            # read in corresponding data: 
            #--------------------------------
            exp_id = f'{solomon_type}101200_diffinitrand{vrp_variant}'

            fp_in = FilePaths(parent_dir+exp_2_file_name, exp_id, setup_directories=False)
            storage = EvaluationStorage(fp_in)
            
            results = storage.get_evaluation_data(exp_id, task_type="alns")                                     ## for all variant datafile 
            if filter_agent is not None:
                results = [r for r in results if r["algorithm"] in filter_agent]
            rows_df = pd.DataFrame(results)
            results_table = pd.pivot_table(rows_df, values=avg_or_min, columns=["algorithm"], index=["cust_number"]).reset_index()

            results_table = pd.DataFrame(results_table.to_records()).reset_index(drop=True)
            results_table = results_table[["cust_number", "randommdp", "classicrw", "dqn", "dqnprob", "hybrid", "vrwmdp"]]      ## for all variant datafile 

            pd.set_option('display.max_columns', None)
            pd.set_option('display.width', 1000)

            #--------------------------------
            # print out dataframe as txt: 
            #--------------------------------
            if print_out_txt:
                with open("alns_evaluation.txt", "a") as file:
                    file.write("=" * 20 + '\n')
                    file.write(exp_id + '\n')
                    file.write("=" * 20 + '\n')
                    results_table_string = results_table.to_string(index=False)
                    file.write(results_table_string + '\n')
                    file.write('\n')

            #--------------------------------
            # plot out line plot 
            #--------------------------------
            title_name = f'ALNS Evaluation ({vrp_variant}, {solomon_type})'
            x_axis_column = "cust_number"
            x_label = 'Instance Size'
            y_label = 'Objective Values'
            columns_to_plot = ['dqn', 'dqnprob', 'vrwmdp', 'randommdp', 'hybrid', 'classicrw']
            
            original_labels = [key for key, value in manual_legend.items() if key in columns_to_plot]
            new_labels = [value for key, value in manual_legend.items() if key in columns_to_plot]
            
            df = pd.DataFrame(results_table)
            df_melted = df.melt(id_vars=x_axis_column, value_vars=columns_to_plot, var_name="Algorithm", value_name="Value")

            sns.set(style='darkgrid')
            plt.figure(figsize=(10, 6))
            ax1 = plt.subplot(111)
            sns.lineplot(data=df_melted, x=x_axis_column, y="Value", hue="Algorithm", marker='o', markersize=8, linewidth=3, hue_order=original_labels)

            plt.subplots_adjust(top=0.9, left=0.1)
            plt.title(title_name, fontsize=16)
            plt.xlabel(x_label, fontsize=16)
            plt.ylabel(y_label, fontsize=16)
            plt.yticks(fontsize=14) 
            plt.xticks(fontsize=16)


            # # Customize the legend. Replace the labels inside the legend
            legend = plt.legend(borderaxespad=0., fontsize=13) #loc='upper left', bbox_to_anchor=(0.06, 0.9), bbox_to_anchor=(1, 1)
            for text, label in zip(legend.get_texts(), new_labels):
                text.set_text(label)
                
            #--------------------------------
            # print out plots in given folder: 
            #--------------------------------
            if not os.path.exists(folder_name):
                os.mkdir(folder_name)

            plt.tight_layout()
            plt.savefig(os.path.join(folder_name, f'{evaluation_type}_{avg_or_min}_{vrp_variant}_solo{solomon_type}'), dpi=300)
            # plt.show()
            plt.close()


In [ ]:
evaluation_type = 'alns-lineplot'
folder_name = f"2.1-{evaluation_type}"
alns_lineplot(evaluation_type, folder_name, avg_or_min='obj_avg')
# alns_lineplot(evaluation_type, folder_name, avg_or_min='obj_min')

### 2.2 lineplot with ci (Preferred One!)
keep all the individual points without aggregating and plot out line with ci

In [ ]:
print_csv_file=False

def alns_lineplot_with_ci(evaluation_type, folder_name, budget='all', avg_or_min='obj_avg'): 

    for solomon_type in solomon_types: 
        for vrp_variant in vrp_variants: 

            #--------------------------------
            # read in corresponding data: 
            #--------------------------------
            exp_id = f'{solomon_type}101200_diffinitrand{vrp_variant}'

            fp_in = FilePaths(parent_dir+exp_2_file_name, exp_id, setup_directories=False)
            storage = EvaluationStorage(fp_in)
            results = storage.get_evaluation_data(exp_id, task_type="alns")  
            results_table = pd.DataFrame(results)
            
            # keep all the budgets or select the specifed one: 
            if budget != 'all': 
                results_table = results_table[(results_table['operator_counter'] == budget)]

            if print_csv_file == True:
                results_table.to_csv("df_table_alns_" + exp_id + ".csv", sep=',', index=False)

            #--------------------------------
            # plot out line plot 
            #--------------------------------
            title_name = f'ALNS Evaluation ({vrp_variant}, {solomon_type}, {avg_or_min}, budget={budget})'
            x_value = "cust_number"
            x_label = 'Instance Size'
            y_value = avg_or_min
            y_label = "Average objective value"
            category = 'algorithm'
            
            original_labels = [key for key, value in manual_legend.items() if key in results_table.algorithm.unique()]
            new_labels = [value for key, value in manual_legend.items() if key in results_table.algorithm.unique()]
            
            sns.set(style='darkgrid')
            plt.figure(figsize=(10, 6))
            sns.lineplot(x=x_value, y=y_value, hue=category, data=results_table, 
                         ci=95, hue_order=original_labels, marker='o', markersize = 8) #linewidth = 1
            
            # plt.title(title_name, fontsize=16)
            plt.xlabel(x_label)
            plt.ylabel(y_label)
            plt.yticks(fontsize=14) 
            plt.xticks(fontsize=16)
            # plt.tick_params(axis='x', labelbottom=False, labeltop=False)
            # plt.tick_params(axis='y', labelleft=False, labelright=False)

            # Customize the legend. Replace the labels inside the legend
            legend = plt.legend(borderaxespad=0., fontsize=13) #loc='upper left', bbox_to_anchor=(0.06, 0.9), bbox_to_anchor=(1, 1)
            for text, label in zip(legend.get_texts(), new_labels):
                text.set_text(label)

            #--------------------------------
            # print out plots in given folder: 
            #--------------------------------
            if not os.path.exists(folder_name):
                os.mkdir(folder_name)

            plt.tight_layout()
            plt.savefig(os.path.join(folder_name, 
                                     f'{evaluation_type}_{avg_or_min}_{budget}_{vrp_variant}_solo{solomon_type}.pdf'), 
                                     bbox_inches='tight', 
                                     dpi=300)

            plt.close()

In [ ]:
evaluation_type = 'alns-lineplot-ci'
folder_name = f"2.2-{evaluation_type}"
budget = 'all'
alns_lineplot_with_ci(evaluation_type, folder_name, budget, avg_or_min="obj_avg")

# budget = 'all'
# alns_lineplot_with_ci(evaluation_type, folder_name, budget, avg_or_min="obj_min")

### Summary Plot (based on 2.2)

In [ ]:
def alns_lineplot_summary(avg_or_min='obj_avg', y_log = False): 

    sns.set(style='darkgrid')
    fig, ax = plt.subplots(figsize=(20,20), nrows=5, ncols=3)

    column_position = -1
    
    for solomon_type in solomon_types: 
        row_position = -1
        column_position += 1

        for vrp_variant in vrp_variants: 
            row_position += 1

            #--------------------------------
            # read in corresponding data: 
            #--------------------------------
            exp_id = f'{solomon_type}101200_diffinitrand{vrp_variant}'

            fp_in = FilePaths(parent_dir+exp_2_file_name, exp_id, setup_directories=False)
            storage = EvaluationStorage(fp_in)
            results = storage.get_evaluation_data(exp_id, task_type="alns")  
            results_table = pd.DataFrame(results)
            
            #--------------------------------
            # plot out line plot 
            #--------------------------------
            x_value = "cust_number"
            x_label = 'Instance Size'
            y_value = avg_or_min
            y_label = "Average Objective Value"
            category = 'algorithm'
            
            original_labels = [key for key, value in manual_legend.items() if key in results_table.algorithm.unique()]
            new_labels = [value for key, value in manual_legend.items() if key in results_table.algorithm.unique()]
            
            # Create a color palette with a consistent order for each unique algorithm
            specific_palette = {old_name: colour for old_name, colour in old_palette.items() if old_name in original_labels}

            sns.lineplot(x=x_value, y=y_value, hue=category, data=results_table, 
                         ax=ax[row_position, column_position], palette=specific_palette,
                         ci=95, hue_order=original_labels, marker='o', markersize = 6) #linewidth = 1
            

            subplot_solomon_type = solomon_type if solomon_type!= 'O' else 'RC'
            ax[row_position, column_position].set_title(f'{vrp_variant}, (type-{subplot_solomon_type})', fontsize=25)
            ax[row_position, column_position].set_xlabel('')
            ax[row_position, column_position].set_ylabel('')
            ax[row_position, column_position].set_xticks([10*x for x in range(2,11)])
            ax[row_position, column_position].tick_params(axis='x', labelsize=16)
            ax[row_position, column_position].tick_params(axis='y', labelsize=12)

            # Set the y-axis to log scale
            if y_log == True:
                ax[row_position, column_position].set_yscale("log")

                if solomon_type == 'O':
                    if vrp_variant == 'TSP':  
                        ax[row_position, column_position].set_ylim(300, 2200) 
                    elif vrp_variant == 'CVRP':  
                        ax[row_position, column_position].set_ylim(900, 2200) 
                    elif vrp_variant == 'HVRP':  
                        ax[row_position, column_position].set_ylim(900, 2300) 
                    elif vrp_variant == 'VRPTW':  
                        ax[row_position, column_position].set_ylim(900, 4000) 
                    elif vrp_variant == 'LRP':  
                        ax[row_position, column_position].set_ylim(2300, 4500) 
            #--------------------------------
            # Customize the legend. Replace the labels inside the legend
            #--------------------------------
            legend = ax[row_position, column_position].get_legend()
            for text, label in zip(legend.get_texts(), new_labels):
                text.set_text(label)

    #--------------------------------
    # print out plots: 
    #--------------------------------
    fig.supxlabel(x_label, fontsize=30)
    fig.supylabel(y_label, fontsize=30)

    fig.subplots_adjust(left=0.1, bottom=0.1)
    
    plt.tight_layout()
    plt.savefig(f"2_ALNS_evaluation_lineplot.pdf", 
                            bbox_inches='tight',
                            dpi=300)
    plt.close()

alns_lineplot_summary(y_log = False)

---
## 3. Extrapolation to Large GH Instance
- Train agent on Solomon up to 50 nodes and apply on Gehring Homeberger instances with up to 1000 nodes. 
- Only in ALNS evaluation, no MDP evaluation 

### Previous Graph

In [ ]:
search_instances = ["02_01","04_01","06_01","08_01","10_01"]
gh_solomon_type = "O"

def percentage_change(col1, col2):
    return ((col2 - col1) / col1) * 100

def large_gh_alns_evaluation(evaluation_type, folder_name, avg_or_min="obj_avg", print_out_graph=False): 
    
    diff_lrw_ran, diff_lrw_crw = [], []
    diff_hybrid_ran, diff_hybrid_crw = [], []

    #--------------------------------
    # read in corresponding data: 
    #--------------------------------
    for vrp_variant in vrp_variants: 
        all_dataframes = []
        for search_instance in search_instances: 
            file_paths = FilePaths(parent_dir, exp_3_file_name, setup_directories=False)
            task_type = "alns"
            results_dir = getattr(FilePaths(file_paths.parent_dir, exp_3_file_name, setup_directories=False), f"{task_type}_results_dir")

            all_results_rows = []
            for results_file in results_dir.iterdir():
                if (vrp_variant in str(results_file)) and (search_instance in str(results_file)):
                    with open(results_file, "rb") as fh:
                        try: 
                            result_rows = json.load(fh)
                            all_results_rows.extend(result_rows)
                        except:
                            continue
            
            results_dataframe = pd.DataFrame(all_results_rows)
            all_dataframes.append(results_dataframe)

        results_table = pd.concat(all_dataframes, ignore_index=True)
        results_table_pivot = pd.pivot_table(results_table, values='obj_avg', columns=["algorithm"], index=["cust_number"]).reset_index()
        

        # compute improvement of ALNS evaluation (LRW versus RAN and CRW)
        diff_lrw_ran.append(-percentage_change(results_table_pivot["random"], results_table_pivot["vrwmdp"]).mean())
        diff_lrw_crw.append(-percentage_change(results_table_pivot["classicrw"], results_table_pivot["vrwmdp"]).mean())
        
        # compute improvement of ALNS evaluation (LRW versus RAN and CRW)
        diff_hybrid_ran.append(-percentage_change(results_table_pivot["random"], results_table_pivot["hybrid"]).mean())
        diff_hybrid_crw.append(-percentage_change(results_table_pivot["classicrw"], results_table_pivot["hybrid"]).mean())
        


        #--------------------------------
        # plot out line plot 
        #--------------------------------
        if print_out_graph == True: 
            title_name = f'ALNS Evaluation large-GH ({vrp_variant}, {gh_solomon_type}, {avg_or_min}, budget={budget})'
            x_value = "cust_number"
            x_label = 'Instance Size'
            y_value = avg_or_min
            y_label = "Average objective value"
            category = 'algorithm'
            
            original_labels = [key for key, value in manual_legend.items() if key in results_table.algorithm.unique()]
            new_labels = [value for key, value in manual_legend.items() if key in results_table.algorithm.unique()]
            
            sns.set(style='darkgrid')
            plt.figure(figsize=(10, 6))
            sns.lineplot(x=x_value, y=y_value, hue=category, data=results_table, ci=95, hue_order=original_labels)
            
            plt.title(title_name, fontsize=16)
            plt.xlabel(x_label, fontsize=16)
            plt.ylabel(y_label, fontsize=16)
            plt.yticks(fontsize=14) 
            plt.xticks(fontsize=16)
            
            # Customize the legend. Replace the labels inside the legend
            legend = plt.legend(borderaxespad=0., fontsize=13) #loc='upper left', bbox_to_anchor=(0.06, 0.9), bbox_to_anchor=(1, 1)
            for text, label in zip(legend.get_texts(), new_labels):
                text.set_text(label)

            #--------------------------------
            # print out plots in given folder: 
            #--------------------------------
            if not os.path.exists(folder_name):
                os.mkdir(folder_name)

            plt.tight_layout()
            plt.savefig(os.path.join(folder_name, f'{evaluation_type}_{avg_or_min}_{budget}_{vrp_variant}_gh{gh_solomon_type}'), dpi=300)
            # plt.show()
            plt.close()
    
    print(f'LRW over RAN and CRW:', np.average(diff_lrw_ran), np.average(diff_lrw_crw))
    print(f'Hybrid over RAN and CRW:', np.average(diff_hybrid_ran), np.average(diff_hybrid_crw))


In [ ]:
evaluation_type = 'alns-lineplot-large-gh'
folder_name = f"3-{evaluation_type}"
large_gh_alns_evaluation(evaluation_type, folder_name)

### Summary Plot

In [ ]:
search_instances = ["02_01","04_01","06_01","08_01","10_01"]
gh_solomon_type = "O"

def large_gh_alns_evaluation_summary(avg_or_min="obj_avg", return_table=False): 
    
    sns.set(style='darkgrid')
    fig = plt.figure(figsize=(20, 8))  #width, height
    rows, columns = 2, 3
    position = 0
    #--------------------------------
    # read in corresponding data: 
    #--------------------------------
    for vrp_variant in vrp_variants: 
        all_dataframes = []
        
        for search_instance in search_instances: 
            file_paths = FilePaths(parent_dir, exp_3_file_name, setup_directories=False)
            task_type = "alns"
            results_dir = getattr(FilePaths(file_paths.parent_dir, exp_3_file_name, setup_directories=False), f"{task_type}_results_dir")

            all_results_rows = []
            for results_file in results_dir.iterdir():
                if (vrp_variant in str(results_file)) and (search_instance in str(results_file)):
                    # print(f'results_file={results_file}')
                    with open(results_file, "rb") as fh:
                        try: 
                            result_rows = json.load(fh)
                            all_results_rows.extend(result_rows)
                        except:
                            continue
            
            results_dataframe = pd.DataFrame(all_results_rows)
            all_dataframes.append(results_dataframe)

        results_table = pd.concat(all_dataframes, ignore_index=True)
        
        #--------------------------------
        # plot out line plot 
        #--------------------------------
        x_value = "cust_number"
        x_label = 'Instance Size'
        y_value = avg_or_min
        y_label = "Average Objective Value"
        category = 'algorithm'
        
        original_labels = [key for key, value in manual_legend.items() if key in results_table.algorithm.unique()]
        new_labels = [value for key, value in manual_legend.items() if key in results_table.algorithm.unique()]
        
        position += 1
        ax = fig.add_subplot(rows, columns, position)

        # Create a color palette with a consistent order for each unique algorithm
        specific_palette = {old_name: colour for old_name, colour in old_palette.items() if old_name in original_labels}

        sns.lineplot(x=x_value, y=y_value, hue=category, data=results_table, ci=95,linewidth=3,
                    hue_order=original_labels, palette=specific_palette)
        
        subplot_solomon_type = gh_solomon_type if gh_solomon_type!= 'O' else 'RC'
        ax.set_title(f'{vrp_variant}, (type-{subplot_solomon_type})', fontsize=20)
        ax.ticklabel_format(axis='y', style='sci', scilimits=(0,4))
        ax.yaxis.major.formatter._useMathText = True  # Enable math text for better control
        ax.tick_params(axis='x', labelsize=16)
        ax.tick_params(axis='y', labelsize=14)
        ax.set_xlabel('')
        ax.set_ylabel('')
        
        # Customize the legend. Replace the labels inside the legend
        legend = plt.legend(borderaxespad=0., fontsize=13) #loc='upper left', bbox_to_anchor=(0.06, 0.9), bbox_to_anchor=(1, 1)
        for text, label in zip(legend.get_texts(), new_labels):
            text.set_text(label)

    #--------------------------------
    # print out plots: 
    #--------------------------------
    fig.supxlabel(x_label, fontsize=30)
    fig.supylabel(y_label, fontsize=30)
    plt.tight_layout()
    plt.savefig(f"3_large_gh_evaluation_lineplot.pdf", 
                            bbox_inches='tight',
                            dpi=300)
    plt.show()

    if return_table == True: 
        return results_table


# large_gh_alns_evaluation_summary()
results_table = large_gh_alns_evaluation_summary(return_table=True)

In [ ]:
results_table

In [ ]:
results_table.agent_seed.unique()

---
## 4. Computational Time

### Previous Graph

In [ ]:
def compute_time_lineplot(evaluation_type, folder_name): 

    for solomon_type in solomon_types: 
        for vrp_variant in vrp_variants: 
            
            # # BUG - in the original file there is no result stored! 
            # if vrp_variant=='VRPTW' and solomon_type=="C":
            #     continue

            #--------------------------------
            # read in corresponding data: 
            #--------------------------------
            exp_id = f'{solomon_type}101200_diffinitrandtimings{vrp_variant}'

            fp_in = FilePaths(parent_dir+exp_4_file_name, exp_id, setup_directories=False)
            storage = EvaluationStorage(fp_in)
            results = storage.get_evaluation_data(exp_id, task_type="alns")  
            results_table = pd.DataFrame(results)
            
            results_table['duration_s'] = results_table['duration_ms'] / 1000

            #--------------------------------
            # plot out line plot 
            #--------------------------------
            title_name = f'Computation Time ({vrp_variant}, {solomon_type}, solomon)'
            x_value = "cust_number"
            x_label = 'Instance Size'
            y_value = "duration_s"
            y_label = "Average compute time (sec)"
            category = 'algorithm'

            original_labels = [key for key, value in manual_legend.items() if key in results_table.algorithm.unique()]
            new_labels = [value for key, value in manual_legend.items() if key in results_table.algorithm.unique()]
                
            sns.set(style='darkgrid')
            plt.figure(figsize=(10, 6))
            sns.lineplot(x=x_value, y=y_value, hue=category, data=results_table, ci=95, hue_order=original_labels)
            
            # plt.title(title_name, fontsize=16)
            plt.xlabel(x_label, fontsize=16)
            plt.ylabel(y_label, fontsize=16)
            plt.yticks(fontsize=14) 
            plt.xticks(fontsize=16)
            
            # Customize the legend. Replace the labels inside the legend
            legend = plt.legend(borderaxespad=0., fontsize=13) #loc='upper left', bbox_to_anchor=(0.06, 0.9), bbox_to_anchor=(1, 1)
            for text, label in zip(legend.get_texts(), new_labels):
                text.set_text(label)

            #--------------------------------
            # print out plots in given folder: 
            #--------------------------------
            if not os.path.exists(folder_name):
                os.mkdir(folder_name)

            plt.tight_layout()
            plt.savefig(os.path.join(folder_name, f'{evaluation_type}_{vrp_variant}_solo{solomon_type}'), dpi=300)
            # plt.show()
            plt.close()

In [ ]:
evaluation_type = 'comput_time_solomon'
folder_name = f"4-{evaluation_type}"
compute_time_lineplot(evaluation_type, folder_name)

### Summary Table

In [ ]:
def compute_time_table_summary(our_model): 

    all_last_rows = []

    for vrp_variant in vrp_variants: 
        for solomon_type in solomon_types: 
            
            if our_model == 'ours': 
                data_type = '101200_diffinitrandtimings'            # our models (DQN, LRW, CRW, RAN)
            elif our_model == 'baseline': 
                data_type = '101200_diffinitrandbaselinetimings'    # baseline model

            #--------------------------------
            # read in corresponding data: 
            #--------------------------------
            exp_id = f'{solomon_type}{data_type}{vrp_variant}'

            fp_in = FilePaths(parent_dir+exp_4_file_name, exp_id, setup_directories=False)
            storage = EvaluationStorage(fp_in)
            results = storage.get_evaluation_data(exp_id, task_type="alns")  
            results_table = pd.DataFrame(results)
            results_table['duration_s'] = results_table['duration_ms'] / 1000

            #--------------------------------
            # assign pivot table and return the last n rows: 
            #--------------------------------
            # results_table_pivot = pd.pivot_table(results_table, values='duration_s', columns=["algorithm"], index=["cust_number"]).reset_index()
            # last_row = results_table_pivot.tail(1)
            # last_row['name'] = f"({solomon_type}, {vrp_variant})"
            last_row = pd.pivot_table(results_table, values='duration_s', columns=["algorithm"], index=["cust_number"]).reset_index().tail(1).assign(type=solomon_type).assign(variant=vrp_variant)
            all_last_rows.append(last_row)

    comput_time = pd.concat(all_last_rows).drop(['cust_number'], axis=1)
    comput_time = comput_time.round(2)
    comput_time.reset_index(drop=True, inplace=True)

    # Define the new column labels
    if our_model == 'ours':
        new_column_labels = ['CRW', 'DQN', 'DQN-prob', 'Hybrid', 'RAN', 'LRW', 'type', 'variant']
        new_column_order = ['variant', 'type', 'DQN', 'DQN-prob', 'LRW', 'Hybrid', 'RAN', 'CRW']
    elif our_model == 'baseline': 
        # new_column_labels = ['DQN-base', 'DQN-prob-base', 'type', 'variant']
        # new_column_order = ['variant', 'type', 'DQN-base', 'DQN-prob-base']
        new_column_labels = ['DQN-base', 'DQN-prob-base']
        new_column_order = new_column_labels
        
    # Reorder the columns and replace labels
    new_comput_time = comput_time.rename(columns=dict(zip(comput_time.columns, new_column_labels)))
    new_comput_time = new_comput_time[new_column_order]

    return new_comput_time

comput_time_1 = compute_time_table_summary(our_model = 'ours')
comput_time_2 = compute_time_table_summary(our_model = 'baseline')

In [ ]:
comput_time = pd.concat([comput_time_1, comput_time_2], axis=1)
comput_time

In [ ]:
latex_code = comput_time.to_latex(index=False, escape=False)
print(latex_code)

---
## 5. Operator Budget Analysis 

### Previous Graphs

In [ ]:
def budget_analysis_boxplot(evaluation_type, folder_name, cust_number, avg_or_min="obj_avg"):
    for solomon_type in solomon_types:
        for vrp_variant in vrp_variants: 
            
            #--------------------------------
            # read in corresponding data: 
            #--------------------------------
            exp_id = f'{solomon_type}101200_diffinitrand{vrp_variant}'

            fp_in = FilePaths(parent_dir+exp_5_file_name, exp_id, setup_directories=False)
            storage = EvaluationStorage(fp_in)
            results = storage.get_evaluation_data(exp_id, task_type="alns")  
            results_table = pd.DataFrame(results)
            
            # cannot read in too many category due to space limitation:
            results_table = results_table[(results_table['algorithm'] != 'randommdp') & (results_table['algorithm'] != 'dqn') & (results_table['algorithm'] != 'classicrw')]
            results_table = results_table[(results_table['cust_number'] == cust_number)]    


            x_value = 'operator_counter' 
            x_label = "Budget"
            y_value = avg_or_min
            y_label = "Average objective value"
            groups = 'algorithm'
                       
            original_labels = [key for key, value in manual_legend.items() if key in results_table.algorithm.unique()]
            new_labels = [value for key, value in manual_legend.items() if key in results_table.algorithm.unique()]
             
            sns.set(style='darkgrid' )
            plt.figure(figsize=(10, 6))
            sns.boxplot(x=x_value, y=y_value, hue=groups, data=results_table, hue_order=original_labels)

            plt.title(f'Budget Analysis (cust={cust_number}, {vrp_variant}, {solomon_type})', fontsize=16)
            plt.xlabel(x_label, fontsize=16)
            plt.ylabel(y_label, fontsize=16)
            plt.yticks(fontsize=14) 
            plt.xticks(fontsize=16)

            #--------------------------------
            # Customize the legend. Replace the labels inside the legend
            #--------------------------------
            # plt.legend(title='Algorithm')
            legend = plt.legend(borderaxespad=0., fontsize=14) #loc='upper left', bbox_to_anchor=(0.06, 0.9), bbox_to_anchor=(1, 1)
            for text, label in zip(legend.get_texts(), new_labels):
                text.set_text(label)

            #--------------------------------
            # print out plots in given folder: 
            #--------------------------------
            if not os.path.exists(folder_name):
                os.mkdir(folder_name)

            plt.tight_layout()
            plt.savefig(os.path.join(folder_name, f'{evaluation_type}_{avg_or_min}_cust{cust_number}_{vrp_variant}_solo{solomon_type}'), dpi=300)
            # plt.show()
            plt.close()

def budget_analysis_lineplot(evaluation_type, folder_name, cust_number, avg_or_min="obj_avg"):
    for solomon_type in solomon_types:
        for vrp_variant in vrp_variants: 
            #--------------------------------
            # read in corresponding data: 
            #--------------------------------
            exp_id = f'{solomon_type}101200_diffinitrand{vrp_variant}'

            fp_in = FilePaths(parent_dir+exp_5_file_name, exp_id, setup_directories=False)
            storage = EvaluationStorage(fp_in)
            results = storage.get_evaluation_data(exp_id, task_type="alns")  
            results_table = pd.DataFrame(results)
            
            results_table = results_table[(results_table['cust_number'] == cust_number)]    


            x_value = 'operator_counter' 
            x_label = "Budget"
            y_value = avg_or_min
            y_label = "Average objective value"
            groups = 'algorithm'

            original_labels = [key for key, value in manual_legend.items() if key in results_table.algorithm.unique()]
            new_labels = [value for key, value in manual_legend.items() if key in results_table.algorithm.unique()]
                         
            sns.set(style='darkgrid')
            plt.figure(figsize=(10, 6))
            sns.lineplot(x=x_value, y=y_value, hue=groups, data=results_table, hue_order=original_labels)
            plt.title(f'Budget Analysis (cust={cust_number}, {vrp_variant}, {solomon_type})', fontsize=16)
            plt.xlabel(x_label, fontsize=16)
            plt.ylabel(y_label, fontsize=16)
            plt.yticks(fontsize=14) 
            plt.xticks(fontsize=16)

            #--------------------------------
            # Customize the legend. Replace the labels inside the legend
            #--------------------------------
            # plt.legend(title='Algorithm')
            legend = plt.legend(borderaxespad=0., fontsize=14) #loc='upper left', bbox_to_anchor=(0.06, 0.9), bbox_to_anchor=(1, 1)
            for text, label in zip(legend.get_texts(), new_labels):
                text.set_text(label)

            #--------------------------------
            # print out plots in given folder: 
            #--------------------------------
            if not os.path.exists(folder_name):
                os.mkdir(folder_name)

            plt.tight_layout()
            plt.savefig(os.path.join(folder_name, f'{evaluation_type}_{avg_or_min}_cust{cust_number}_{vrp_variant}_solo{solomon_type}'), dpi=300)
            # plt.show()
            plt.close()

In [ ]:
# Box plot (only 3 models)
evaluation_type = "budget-analysis" 
folder_name = f"5.1-{evaluation_type}-boxplot" 
cust_number = 100
budget_analysis_boxplot(evaluation_type, folder_name, cust_number, avg_or_min="obj_avg")

# Line plot (all 6 models)
evaluation_type = "budget-analysis" 
folder_name = f"5.2-{evaluation_type}-lineplot"
cust_number = 100
budget_analysis_lineplot(evaluation_type, folder_name, cust_number, avg_or_min="obj_avg")

### Summary Plot

In [ ]:
def budget_analysis_lineplot_summary(cust_number, avg_or_min="obj_avg"):
    
    sns.set(style='darkgrid')
    fig, ax = plt.subplots(figsize=(20,20), nrows=5, ncols=3)

    column_position = -1

    for solomon_type in solomon_types:
        row_position = -1
        column_position += 1

        for vrp_variant in vrp_variants: 
            row_position += 1
      
            #--------------------------------
            # read in corresponding data: 
            #--------------------------------
            exp_id = f'{solomon_type}101200_diffinitrand{vrp_variant}'

            fp_in = FilePaths(parent_dir+exp_5_file_name, exp_id, setup_directories=False)
            storage = EvaluationStorage(fp_in)
            results = storage.get_evaluation_data(exp_id, task_type="alns")  
            results_table = pd.DataFrame(results)
            
            results_table = results_table[(results_table['cust_number'] == cust_number)]    

            x_value = 'operator_counter' 
            x_label = "Operator Budget Size"
            y_value = avg_or_min
            y_label = "Average Objective Value"
            groups = 'algorithm'

            original_labels = [key for key, value in manual_legend.items() if key in results_table.algorithm.unique()]
            new_labels = [value for key, value in manual_legend.items() if key in results_table.algorithm.unique()]
                         
            # Create a color palette with a consistent order for each unique algorithm
            specific_palette = {old_name: colour for old_name, colour in old_palette.items() if old_name in original_labels}

            sns.lineplot(x=x_value, y=y_value, hue=groups, data=results_table, 
                         ax=ax[row_position, column_position], palette=specific_palette,
                         hue_order=original_labels)

            subplot_solomon_type = solomon_type if solomon_type!= 'O' else 'RC'
            ax[row_position, column_position].set_title(f'{vrp_variant}, (type-{subplot_solomon_type})', fontsize=25)
            ax[row_position, column_position].set_xlabel('')
            ax[row_position, column_position].set_ylabel('')
            ax[row_position, column_position].set_xticks([10*x for x in range(2,11)])
            ax[row_position, column_position].tick_params(axis='x', labelsize=16)
            ax[row_position, column_position].tick_params(axis='y', labelsize=12)

            #--------------------------------
            # Customize the legend. Replace the labels inside the legend
            #--------------------------------
            legend = ax[row_position, column_position].get_legend()
            for text, label in zip(legend.get_texts(), new_labels):
                text.set_text(label)

    #--------------------------------
    # print out plots in given folder: 
    #--------------------------------
    fig.supxlabel(x_label, fontsize=30)
    fig.supylabel(y_label, fontsize=30)

    fig.subplots_adjust(left=0.1, bottom=0.1)
    
    plt.tight_layout()
    plt.savefig(f"5_budget_analysis_summary.pdf", 
                            bbox_inches='tight',
                            dpi=300)
    plt.close()


budget_analysis_lineplot_summary(cust_number=100)

---
## 6. ALNS Baseline Comparison

- Contains the baseline results for all 15 problems, from competitor paper
- Two additional agents to be added to the plots, named "dqnbaseline" (for the MDP and ALNS) and "dqnbaselineprob" (for ALNS only)

### 6.1 MDP barplot (with new agent)

In [ ]:
def mdp_barplot_baseline(evaluation_type, folder_name):

    for solomon_type in solomon_types: 
        for vrp_variant in vrp_variants: 

            #--------------------------------
            # read in corresponding data: 
            #--------------------------------
            # baseline data: 
            exp_id_1 = f'{solomon_type}101200_diffinitrandbaseline{vrp_variant}' 
            fp_in_1 = FilePaths(parent_dir + exp_6_file_name_1, exp_id_1, setup_directories=False)
            results_1 = EvaluationStorage(fp_in_1).get_evaluation_data(exp_id_1, task_type="eval") 
            rows_df_1 = pd.DataFrame(results_1)

            # original mdp (R and C) or (O): 
            if solomon_type == "O":
                exp_id_2 = f'{solomon_type}101200_diffinitrandredone{vrp_variant}'
                fp_in_2 = FilePaths(parent_dir + exp_6_file_name_3, exp_id_2, setup_directories=False)
            else:
                exp_id_2 = f'{solomon_type}101200_diffinitrand{vrp_variant}' 
                fp_in_2 = FilePaths(parent_dir + exp_6_file_name_2, exp_id_2, setup_directories=False)
            results_2 = EvaluationStorage(fp_in_2).get_evaluation_data(exp_id_2, task_type="eval") 
            rows_df_2 = pd.DataFrame(results_2)
            
            rows_df = pd.concat([rows_df_1, rows_df_2], ignore_index=True)
            
            results_table = pd.pivot_table(rows_df, values='cummulative_reward', columns=["algorithm"], index=["cust_number"]).reset_index()
        
            pd.set_option('display.max_columns', None)
            pd.set_option('display.width', 1000)


            #--------------------------------
            # plot out bar plot/histogram:   
            #--------------------------------
            title_name = f'MDP Training with baseline ({vrp_variant}, {solomon_type})'
            x_axis_column = "cust_number"
            x_label = 'Instance Size'
            y_label = 'Cum. Rewards'
            columns_to_plot = ['cust_number', 'dqn', 'dqnbaseline', 'vrwmdp', 'randommdp']

            original_labels = [key for key, value in manual_legend.items() if key in rows_df.algorithm.unique()]
            new_labels = [value for key, value in manual_legend.items() if key in rows_df.algorithm.unique()]
            
            df = pd.DataFrame(results_table)
            df_melted = df.melt(id_vars=x_axis_column, value_vars=columns_to_plot, var_name='method', value_name='value')  # Melt the DataFrame to convert it to the long format

            sns.set(style='darkgrid')
            plt.figure(figsize=(10, 6))
            ax = plt.subplot(111)
            sns.barplot(x=x_axis_column, y='value', hue='method', data=df_melted, ci=None, ax=ax, hue_order=original_labels)

            plt.subplots_adjust(top=0.9, left=0.1)
            plt.title(title_name, fontsize=16)
            plt.xlabel(x_label, fontsize=16)
            plt.ylabel(y_label, fontsize=16)
            plt.yticks(fontsize=14)
        
            new_x_labels = [10*x for x in range(2,11)]   
            plt.xticks(range(len(new_x_labels)), new_x_labels, fontsize=16)

            #--------------------------------
            # Customize the legend. Replace the labels inside the legend
            #--------------------------------
            legend = plt.legend(borderaxespad=0., fontsize=14) #loc='upper left', bbox_to_anchor=(0.06, 0.9), bbox_to_anchor=(1, 1)
            for text, label in zip(legend.get_texts(), new_labels):
                text.set_text(label)

            #--------------------------------
            # print out plots in given folder: 
            #--------------------------------
            if not os.path.exists(folder_name):
                os.mkdir(folder_name)

            plt.tight_layout()
            plt.savefig(os.path.join(folder_name, f'{evaluation_type}_{vrp_variant}_solo{solomon_type}'), dpi=300)
            # plt.show()
            plt.close()

In [ ]:
evaluation_type = 'mdp-baseline-barplot'
folder_name = "6.1-" + evaluation_type
mdp_barplot_baseline(evaluation_type, folder_name)

### 6.2 MDP boxplot (with new agent)

In [ ]:
def mdp_boxplot_baseline(evaluation_type, folder_name): 

    for solomon_type in solomon_types: 
        for vrp_variant in vrp_variants: 
        
            #--------------------------------
            # read in data (baseline + original)
            #--------------------------------
            # baseline data: 
            exp_id_1 = f'{solomon_type}101200_diffinitrandbaseline{vrp_variant}' 
            fp_in_1 = FilePaths(parent_dir + exp_6_file_name_1, exp_id_1, setup_directories=False)
            results_1 = EvaluationStorage(fp_in_1).get_evaluation_data(exp_id_1, task_type="eval") 
            rows_df_1 = pd.DataFrame(results_1)

            # original mdp (R and C) or (O): 
            if solomon_type == "O":
                exp_id_2 = f'{solomon_type}101200_diffinitrandredone{vrp_variant}'
                fp_in_2 = FilePaths(parent_dir + exp_6_file_name_3, exp_id_2, setup_directories=False)
            else:
                exp_id_2 = f'{solomon_type}101200_diffinitrand{vrp_variant}' 
                fp_in_2 = FilePaths(parent_dir + exp_6_file_name_2, exp_id_2, setup_directories=False)
            results_2 = EvaluationStorage(fp_in_2).get_evaluation_data(exp_id_2, task_type="eval") 
            rows_df_2 = pd.DataFrame(results_2)
            
            rows_df = pd.concat([rows_df_1, rows_df_2], ignore_index=True)

            #--------------------------------
            # plot out box plot:   
            #--------------------------------
            title_name = f'MDP Training with baseline ({vrp_variant}, {solomon_type})'
            x_value = "cust_number"
            x_label = 'Instance Size'
            y_value = 'cummulative_reward'
            y_label = 'Cum. Rewards'
            category = "algorithm"            
            
            original_labels = [key for key, value in manual_legend.items() if key in rows_df.algorithm.unique()]
            new_labels = [value for key, value in manual_legend.items() if key in rows_df.algorithm.unique()]
            
            sns.set(style='darkgrid')
            plt.figure(figsize=(10, 6))
            ax = plt.subplot(111)
            sns.boxplot(x=x_value, y=y_value, hue=category, data=rows_df, ax=ax, hue_order=original_labels)

            plt.subplots_adjust(top=0.9, left=0.1)
            plt.title(title_name, fontsize=16)
            plt.xlabel(x_label, fontsize=14)
            plt.ylabel(y_label, fontsize=14)
            # plt.ylim([0, 2500])

            #--------------------------------
            # Customize the legend. Replace the labels inside the legend
            #--------------------------------
            legend = plt.legend(borderaxespad=0., fontsize=13) #loc='upper left', bbox_to_anchor=(0.06, 0.9), bbox_to_anchor=(1, 1)
            for text, label in zip(legend.get_texts(), new_labels):
                text.set_text(label)

            if not os.path.exists(folder_name):
                os.mkdir(folder_name)

            plt.tight_layout()
            plt.savefig(os.path.join(folder_name, f'{evaluation_type}_{vrp_variant}_solo{solomon_type}'), dpi=300)
            # plt.show()
            plt.close()

In [ ]:
evaluation_type = 'mdp-baseline-boxplot'
folder_name = "6.2-" + evaluation_type
mdp_boxplot_baseline(evaluation_type, folder_name)

### 6.3 ALNS lineplot (with 2 new agents)

In [ ]:
def alns_lineplot_with_ci(evaluation_type, folder_name, budget='all', avg_or_min='obj_avg'): 

    for solomon_type in solomon_types: 
        for vrp_variant in vrp_variants: 
            #--------------------------------
            # read in corresponding data: 
            #--------------------------------
            # baseline data: 
            exp_id_1 = f'{solomon_type}101200_diffinitrandbaseline{vrp_variant}' 
            fp_in_1 = FilePaths(parent_dir + exp_6_file_name_1, exp_id_1, setup_directories=False)
            results_1 = EvaluationStorage(fp_in_1).get_evaluation_data(exp_id_1, task_type="alns") 
            rows_df_1 = pd.DataFrame(results_1)

            # original mdp (R and C) or (O): 
            if solomon_type == "O":
                exp_id_2 = f'{solomon_type}101200_diffinitrandredone{vrp_variant}'
                fp_in_2 = FilePaths(parent_dir + exp_6_file_name_3, exp_id_2, setup_directories=False)
            else:
                exp_id_2 = f'{solomon_type}101200_diffinitrand{vrp_variant}' 
                fp_in_2 = FilePaths(parent_dir + exp_6_file_name_2, exp_id_2, setup_directories=False)
            results_2 = EvaluationStorage(fp_in_2).get_evaluation_data(exp_id_2, task_type="alns") 
            rows_df_2 = pd.DataFrame(results_2)
            
            results_table = pd.concat([rows_df_1, rows_df_2], ignore_index=True)
            

            # keep all the budgets or select the specifed one: 
            if budget != 'all': 
                results_table = results_table[(results_table['operator_counter'] == budget)]
            
            # remove additional lines due to space limitation: 
            results_table = results_table[(results_table['algorithm'] != 'dqn')]
            results_table = results_table[(results_table['algorithm'] != 'randommdp')]
            results_table = results_table[(results_table['algorithm'] != 'classicrw')]
        
        
            #--------------------------------
            # plot out line plot 
            #----------------------- ---------
            title_name = f'{vrp_variant}, (type {solomon_type})'
            x_value = "cust_number"
            x_label = 'Instance Size'
            y_value = avg_or_min
            y_label = "Average objective value"
            category = 'algorithm'
            
            original_labels = [key for key, value in manual_legend.items() if key in results_table.algorithm.unique()]
            new_labels = [value for key, value in manual_legend.items() if key in results_table.algorithm.unique()]
            
            sns.set(style='darkgrid')
            plt.figure(figsize=(10, 6))
            sns.lineplot(x=x_value, y=y_value, hue=category, data=results_table, ci=95, hue_order=original_labels)
            
            # plt.title(title_name, fontsize=16)
            plt.xlabel(x_label, fontsize=16)
            plt.ylabel(y_label, fontsize=16)
            plt.yticks(fontsize=14) 
            plt.xticks(fontsize=16)
            
            #--------------------------------
            # Customize the legend. Replace the labels inside the legend
            #--------------------------------
            legend = plt.legend(borderaxespad=0., fontsize=13) #loc='upper left', bbox_to_anchor=(0.06, 0.9), bbox_to_anchor=(1, 1)
            for text, label in zip(legend.get_texts(), new_labels):
                text.set_text(label)

            #--------------------------------
            # print out plots in given folder: 
            #--------------------------------
            if not os.path.exists(folder_name):
                os.mkdir(folder_name)

            plt.tight_layout()
            plt.savefig(os.path.join(folder_name, f'{evaluation_type}_{avg_or_min}_{budget}_{vrp_variant}_solo{solomon_type}'), dpi=300)
            # plt.show()
            plt.close()

In [ ]:
evaluation_type = 'alns-baseline-lineplot-ci'
folder_name = f"6.3-{evaluation_type}"
alns_lineplot_with_ci(evaluation_type, folder_name, budget='all', avg_or_min="obj_avg")
# alns_lineplot_with_ci(evaluation_type, folder_name, budget='all', avg_or_min="obj_min")

### Summary: Baseline MDP (merge with 1.2)

In [ ]:
def mdp_baseline_barplot_summary_plot(): 

    sns.set(style='darkgrid')
    fig, ax = plt.subplots(figsize=(20,20), nrows=5, ncols=3)

    column_position = -1
    
    for solomon_type in solomon_types: 
        row_position = -1
        column_position += 1
        
        for vrp_variant in vrp_variants: 
            row_position += 1

            #--------------------------------
            # read in corresponding data: 
            #--------------------------------
            # baseline data: 
            exp_id_1 = f'{solomon_type}101200_diffinitrandbaseline{vrp_variant}' 
            fp_in_1 = FilePaths(parent_dir + exp_6_file_name_1, exp_id_1, setup_directories=False)
            results_1 = EvaluationStorage(fp_in_1).get_evaluation_data(exp_id_1, task_type="eval") 
            rows_df_1 = pd.DataFrame(results_1)

            # original mdp (R and C) or (O): 
            if solomon_type == "O":
                exp_id_2 = f'{solomon_type}101200_diffinitrandredone{vrp_variant}'
                fp_in_2 = FilePaths(parent_dir + exp_6_file_name_3, exp_id_2, setup_directories=False)
            else:
                exp_id_2 = f'{solomon_type}101200_diffinitrand{vrp_variant}' 
                fp_in_2 = FilePaths(parent_dir + exp_6_file_name_2, exp_id_2, setup_directories=False)
            results_2 = EvaluationStorage(fp_in_2).get_evaluation_data(exp_id_2, task_type="eval") 
            rows_df_2 = pd.DataFrame(results_2)
            
            rows_df = pd.concat([rows_df_1, rows_df_2], ignore_index=True)
            
            pd.set_option('display.max_columns', None)
            pd.set_option('display.width', 1000)
            
            #--------------------------------
            # plot out box plot:   
            #--------------------------------
            title_name = f'MDP Evaluation ({vrp_variant}, {solomon_type})'
            x_value = "cust_number"
            x_label = 'Instance Size'
            y_value = 'cummulative_reward'
            y_label = 'Cum. Rewards'
            category = "algorithm"            
            
            original_labels = [key for key, value in manual_legend.items() if key in rows_df.algorithm.unique()]
            new_labels = [value for key, value in manual_legend.items() if key in rows_df.algorithm.unique()]
            
            # Create a color palette with a consistent order for each unique algorithm
            specific_palette = {old_name: colour for old_name, colour in old_palette.items() if old_name in original_labels}

            sns.barplot(x=x_value, y=y_value, hue=category, data=rows_df, palette=specific_palette,
                        ax=ax[row_position, column_position], hue_order=original_labels, ci=95, errcolor='black')

            subplot_solomon_type = solomon_type if solomon_type!= 'O' else 'RC'
            ax[row_position, column_position].set_title(f'{vrp_variant}, (type-{subplot_solomon_type})', fontsize=25)
            ax[row_position, column_position].set_xlabel('')
            ax[row_position, column_position].set_ylabel('')
            ax[row_position, column_position].tick_params(axis='x', labelsize=16)
            ax[row_position, column_position].tick_params(axis='y', labelsize=12)

            #--------------------------------
            # Customize the legend. Replace the labels inside the legend
            #--------------------------------
            legend = ax[row_position, column_position].get_legend()
            for text, label in zip(legend.get_texts(), new_labels):
                text.set_text(label)

    # plt.setp(ax[-1, :], xlabel=x_label)
    # plt.setp(ax[:, 0], ylabel=y_label)
    fig.supxlabel(x_label, fontsize=30)
    fig.supylabel(y_label, fontsize=30)

    fig.subplots_adjust(left=0.1, bottom=0.1)

    plt.tight_layout()
    plt.savefig(f"1_MDP_evaluation_barplot_baseline.pdf", 
                            bbox_inches='tight',
                            dpi=300)
    plt.close()

mdp_baseline_barplot_summary_plot()

### Summary: Baseline ALNS (switch to table!)

In [ ]:
def alns_table_ratio_summary(): 

    all_rows = []

    for vrp_variant in vrp_variants: 
        for solomon_type in solomon_types: 
        
            #--------------------------------
            # read in corresponding data: 
            #--------------------------------
            # baseline data: 
            exp_id_1 = f'{solomon_type}101200_diffinitrandbaseline{vrp_variant}' 
            fp_in_1 = FilePaths(parent_dir + exp_6_file_name_1, exp_id_1, setup_directories=False)
            results_1 = EvaluationStorage(fp_in_1).get_evaluation_data(exp_id_1, task_type="alns") 
            rows_df_1 = pd.DataFrame(results_1)

            # original mdp (R and C) or (O): 
            if solomon_type == "O":
                exp_id_2 = f'{solomon_type}101200_diffinitrandredone{vrp_variant}'
                fp_in_2 = FilePaths(parent_dir + exp_6_file_name_3, exp_id_2, setup_directories=False)
            else:
                exp_id_2 = f'{solomon_type}101200_diffinitrand{vrp_variant}' 
                fp_in_2 = FilePaths(parent_dir + exp_6_file_name_2, exp_id_2, setup_directories=False)
            results_2 = EvaluationStorage(fp_in_2).get_evaluation_data(exp_id_2, task_type="alns") 
            rows_df_2 = pd.DataFrame(results_2)
            
            results_table = pd.concat([rows_df_1, rows_df_2], ignore_index=True)
            
            # pivot table
            results_table_pivot = pd.pivot_table(results_table, values='obj_avg', columns=["algorithm"], index=["cust_number"]).reset_index()
            
            # add a gap column: 
            # results_table_pivot['gap'] = (results_table_pivot['dqnbaselineprob'] - results_table_pivot['dqnprob']) / results_table_pivot['dqnprob']
            results_table_pivot['gap'] = (results_table_pivot['dqnbaselineprob'] - results_table_pivot['dqnprob']) / results_table_pivot['dqnbaselineprob']
            
            the_row = results_table_pivot['gap'].tolist()
            print(the_row + [solomon_type, vrp_variant])
            all_rows.append(the_row + [solomon_type, vrp_variant])

    return all_rows

all_rows = alns_table_ratio_summary()

In [ ]:
df = pd.DataFrame(all_rows)
df

In [ ]:
# remove the last two columns: 
df_removed_columns = df.iloc[:, :-2]

# converge to percentage
df_percentage = df_removed_columns.apply(lambda x: x * 100).round(2)

# add column titles: 
df_percentage.columns = [10*x for x in range(2,11)] #+ ['instance', 'problem']
df_percentage

In [ ]:
formatter = lambda x: f'{x:.2f}\\%'
latex_code = df_percentage.to_latex(index=False, escape=False, formatters=[formatter]*len(df_percentage.columns))
print(latex_code)

### Summary: Significance Testing

In [ ]:
from scipy.stats import mannwhitneyu
from scipy.stats import f_oneway

def significance_testing_summary(cust_number): 

    for vrp_variant in vrp_variants: 
        for solomon_type in solomon_types: 
        
            #--------------------------------
            # read in corresponding data: 
            #--------------------------------
            # baseline data: 
            exp_id_1 = f'{solomon_type}101200_diffinitrandbaseline{vrp_variant}' 
            fp_in_1 = FilePaths(parent_dir + exp_6_file_name_1, exp_id_1, setup_directories=False)
            results_1 = EvaluationStorage(fp_in_1).get_evaluation_data(exp_id_1, task_type="alns") 
            rows_df_1 = pd.DataFrame(results_1)

            # original mdp (R and C) or (O): 
            if solomon_type == "O":
                exp_id_2 = f'{solomon_type}101200_diffinitrandredone{vrp_variant}'
                fp_in_2 = FilePaths(parent_dir + exp_6_file_name_3, exp_id_2, setup_directories=False)
            else:
                exp_id_2 = f'{solomon_type}101200_diffinitrand{vrp_variant}' 
                fp_in_2 = FilePaths(parent_dir + exp_6_file_name_2, exp_id_2, setup_directories=False)
            results_2 = EvaluationStorage(fp_in_2).get_evaluation_data(exp_id_2, task_type="alns") 
            rows_df_2 = pd.DataFrame(results_2)
            
            df = pd.concat([rows_df_1, rows_df_2], ignore_index=True)
            
            #--------------------------------
            # compute significance
            #--------------------------------
            # Extract two groups of objective values for "A" and "B"
            group_A = df[(df['algorithm'] == 'dqnbaselineprob') & (df['cust_number'] == cust_number)]['obj_avg']
            group_B = df[(df['algorithm'] == 'dqnprob') & (df['cust_number'] == cust_number)]['obj_avg']
        
            # Perform Mann-Whitney U test
            statistic, p_value = mannwhitneyu(group_A, group_B)
            # print(f'Mann-Whitney U statistic: {statistic}')
            # print(f'p-value: {p_value}')

            # Check for significance based on the p-value
            alpha = 0.05  # Set your significance level

            stat, p_one_way = f_oneway(group_A, group_B)

            if (p_value < alpha) and (p_one_way < alpha):
                print(f'{vrp_variant}, {solomon_type}, {round(p_value,3)}, {round(p_one_way,3)}, Different') # - The two groups have significantly different distributions.
            else:
                print(f'{vrp_variant}, {solomon_type}, {round(p_value,3)}, {round(p_one_way,3)}, Same') # - There is no significant difference between the two groups.
    

for input_cust_number in [100-10*x for x in range(9)]:
    print('------------------')
    print(input_cust_number) 
    significance_testing_summary(input_cust_number)

In [ ]:
format(float(4.489749347530464e-12), '.15f')

In [ ]:
len(group_A.tolist())

In [ ]:
group_B

---
## 7. Clarks-Wright Initial Solution Analysis

### 7.1 Noise Analysis

In [ ]:
def matrix_transpose(matrix):
    return [list(x) for x in np.array(matrix).transpose(1,0)]

def plot_noise_analysis(folder_name, node_list, node_size): 
    sns.set(style='darkgrid')
    plt.figure(figsize=(10, 6))

    for line in matrix_transpose(node_list):
        sns.lineplot(data=line, marker='o', markersize=8, linewidth=3)

    custom_ticks = [x for x in range(6)]
    custom_labels = [0.05, 0.1, 0.2, 0.3, 0.4, 0.5]
    plt.xticks(custom_ticks, custom_labels, fontsize=16)
    plt.yticks(fontsize=16)

    plt.ylim(0, 100)  # Set the lower and upper limits for the y-axis

    plt.xlabel('noise level', fontsize=20)
    plt.ylabel('duplicated tours (%)', fontsize=20)
    # plt.title(f"ALNS Performance Curve ({node_size} nodes)", fontsize=24)
    
    colors = ['purple', 'red', 'green', 'black', 'blue']
    lines = [Line2D([0], [0], color=c, linewidth=3, linestyle='-') for c in colors]
    labels = ['TSP', 'CVRP', 'HVRP', 'VRPTW', 'LRP']
    plt.legend(lines, labels, fontsize=16)
    # plt.legend(borderaxespad=0., fontsize=13)
    
    #--------------------------------
    # print out plots in given folder: 
    #--------------------------------
    if not os.path.exists(folder_name):
        os.mkdir(folder_name)

    plt.tight_layout()
    plt.savefig(os.path.join(folder_name, f'noise_level_analysis_{node_size}_nodes.png'))
    # plt.show()
    plt.close()

In [ ]:
node50 = [
    [3.70, 5.70, 5.70, 5.70, 12.40],
    [0.00, 0.00, 0.00, 0.00, 0.00],
    [0.00, 0.00, 0.00, 0.00, 0.00],
    [0.00, 0.00, 0.00, 0.00, 0.00],
    [0.00, 0.00, 0.00, 0.00, 0.00],
    [0.00, 0.00, 0.00, 0.00, 0.00]
]
node40 = [
    [6.30, 9.70, 9.70, 9.70, 42.50],
    [0.00, 0.00, 0.00, 0.00, 0.00],
    [0.00, 0.00, 0.00, 0.00, 0.00],
    [0.00, 0.00, 0.00, 0.00, 0.00],
    [0.00, 0.00, 0.00, 0.00, 0.00],
    [0.00, 0.00, 0.00, 0.00, 0.00]
]
node30 = [
    [41.10, 62.70, 62.70, 62.70, 84.90],
    [0.20, 1.00, 1.00, 1.00, 24.80],
    [0.00, 0.00, 0.00, 0.00, 0.00],
    [0.00, 0.00, 0.00, 0.00, 0.00],
    [0.00, 0.00, 0.00, 0.00, 0.00],
    [0.00, 0.00, 0.00, 0.00, 0.00]
]
node20 = [
    [82.70, 92.10, 92.10, 92.10, 99.70],
    [30.20, 48.50, 48.50, 48.50, 94.50],
    [0.40, 1.40, 1.40, 1.40, 27.80],
    [0.00, 0.20, 0.20, 0.20, 0.80],
    [0.00, 0.00, 0.00, 0.00, 0.00],
    [0.00, 0.00, 0.00, 0.00, 0.00]
]


evaluation_type = 'noise-anlaysis'
folder_name = folder_name = f"7.1-{evaluation_type}"

plot_noise_analysis(folder_name, node20, 20)
plot_noise_analysis(folder_name, node30, 30)
plot_noise_analysis(folder_name, node40, 40)
plot_noise_analysis(folder_name, node50, 50)

### 7.2 + 7.3 MDP barplot + boxplot
Use different initial solutions from Clarks-Wright construction algorithm. Individual points not aggregated by pivot table. 

In [ ]:
clarks_wright_solomon_types = ["O"]

def mdp_clarks_wright_boxplot(evaluation_type, folder_name, plot_type="boxplot"): 

    for solomon_type in clarks_wright_solomon_types: 
        for vrp_variant in vrp_variants: 
        
            #--------------------------------
            # read in corresponding data: 
            #--------------------------------
            exp_id = f'{solomon_type}101200_diffinitcwnoise{vrp_variant}'
            fp_in = FilePaths(parent_dir + exp_7_file_name, exp_id, setup_directories=False)
            storage = EvaluationStorage(fp_in)
            results = storage.get_evaluation_data(exp_id, task_type="eval") 
            results = [r for r in results if r["is_best_hyps"]]
            rows_df = pd.DataFrame(results)

            #--------------------------------
            # plot out box plot:   
            #--------------------------------
            title_name = f'MDP Clarks-Wright ({vrp_variant}, {solomon_type})'
            x_value = "cust_number"
            x_label = 'Instance Size'
            y_value = 'cummulative_reward'
            y_label = 'Cum. Rewards'
            category = "algorithm"            
            
            original_labels = [key for key, value in manual_legend.items() if key in rows_df.algorithm.unique()]
            new_labels = [value for key, value in manual_legend.items() if key in rows_df.algorithm.unique()]
            
            sns.set(style='darkgrid')
            plt.figure(figsize=(10, 6))
            ax = plt.subplot(111)
            
            if plot_type == "boxplot":
                sns.boxplot(x=x_value, y=y_value, hue=category, data=rows_df, ax=ax, hue_order=original_labels)
            elif plot_type == "barplot":
                sns.barplot(x=x_value, y=y_value, hue=category, data=rows_df, ax=ax, hue_order=original_labels)
            else:
                raise ValueError(f'plot_type must be boxplot or barplot')
            
            plt.subplots_adjust(top=0.9, left=0.1)
            plt.title(title_name, fontsize=16)
            plt.xlabel(x_label, fontsize=14)
            plt.ylabel(y_label, fontsize=14)

            #--------------------------------
            # Customize the legend. Replace the labels inside the legend
            #--------------------------------
            legend = plt.legend(borderaxespad=0., fontsize=14) #loc='upper left', bbox_to_anchor=(0.06, 0.9), bbox_to_anchor=(1, 1)
            for text, label in zip(legend.get_texts(), new_labels):
                text.set_text(label)
            
            if not os.path.exists(folder_name):
                os.mkdir(folder_name)

            plt.tight_layout()
            plt.savefig(os.path.join(folder_name, f'{evaluation_type}_{vrp_variant}_solo{solomon_type}'), dpi=300)
            # plt.show()
            plt.close()


In [ ]:
plot_type = "barplot"
evaluation_type = f'mdp-clarkswright-{plot_type}'
folder_name = f"7.2-{evaluation_type}"
mdp_clarks_wright_boxplot(evaluation_type, folder_name, plot_type = plot_type)


plot_type = "boxplot"
evaluation_type = f'mdp-clarkswright-{plot_type}'
folder_name = f"7.3-{evaluation_type}"
mdp_clarks_wright_boxplot(evaluation_type, folder_name, plot_type = plot_type)

### 7.4 ALNS evaluation 
Use different initial solutions from Clarks-Wright construction algorithm. Individual points not aggregated by pivot table. 

In [ ]:
clarks_wright_solomon_types = ["O"]

def alns_lineplot_with_ci(evaluation_type, folder_name, budget='all', avg_or_min='obj_avg'): 

    for solomon_type in clarks_wright_solomon_types: 
        for vrp_variant in vrp_variants: 
            
            #--------------------------------
            # read in corresponding data: 
            #--------------------------------
            exp_id = f'{solomon_type}101200_diffinitcwnoise{vrp_variant}'

            fp_in = FilePaths(parent_dir + exp_7_file_name, exp_id, setup_directories=False)
            storage = EvaluationStorage(fp_in)
            results = storage.get_evaluation_data(exp_id, task_type="alns")  
            results_table = pd.DataFrame(results)
            
            # keep all the budgets or select the specifed one: 
            if budget != 'all': 
                results_table = results_table[(results_table['operator_counter'] == budget)]


            #--------------------------------
            # plot out line plot 
            #--------------------------------
            title_name = f'ALNS Evaluation Clarks-Wright ({vrp_variant}, {solomon_type}, {avg_or_min}, budget={budget})'
            x_value = "cust_number"
            x_label = 'Instance Size'
            y_value = avg_or_min
            y_label = "Average objective value"
            category = 'algorithm'
            
            original_labels = [key for key, value in manual_legend.items() if key in results_table.algorithm.unique()]
            new_labels = [value for key, value in manual_legend.items() if key in results_table.algorithm.unique()]
            
            sns.set(style='darkgrid')
            plt.figure(figsize=(10, 6))
            sns.lineplot(x=x_value, y=y_value, hue=category, data=results_table, ci=95, hue_order=original_labels)
            
            plt.title(title_name, fontsize=16)
            plt.xlabel(x_label, fontsize=16)
            plt.ylabel(y_label, fontsize=16)
            plt.yticks(fontsize=14) 
            plt.xticks(fontsize=16)
            
            # Customize the legend. Replace the labels inside the legend
            legend = plt.legend(borderaxespad=0., fontsize=13) #loc='upper left', bbox_to_anchor=(0.06, 0.9), bbox_to_anchor=(1, 1)
            for text, label in zip(legend.get_texts(), new_labels):
                text.set_text(label)

            #--------------------------------
            # print out plots in given folder: 
            #--------------------------------
            if not os.path.exists(folder_name):
                os.mkdir(folder_name)

            plt.tight_layout()
            plt.savefig(os.path.join(folder_name, f'{evaluation_type}_{avg_or_min}_{budget}_{vrp_variant}_solo{solomon_type}'), dpi=300)
            # plt.show()
            plt.close()

In [ ]:
evaluation_type = 'alns-lineplot-clarks-wright'
folder_name = f"7.4-{evaluation_type}"
budget = 'all'
alns_lineplot_with_ci(evaluation_type, folder_name, budget, avg_or_min="obj_avg")

### Summary: MDP evaluation

In [ ]:
clarks_wright_solomon_types = ["O"]

def mdp_clarks_wright_boxplot(plot_type="boxplot"): 

    sns.set(style='darkgrid')
    fig = plt.figure(figsize=(20, 8))  #width, height
    rows, columns = 2, 3
    position = 0

    for solomon_type in clarks_wright_solomon_types: 
        for vrp_variant in vrp_variants: 
        
            #--------------------------------
            # read in corresponding data: 
            #--------------------------------
            exp_id = f'{solomon_type}101200_diffinitcwnoise{vrp_variant}'
            fp_in = FilePaths(parent_dir + exp_7_file_name, exp_id, setup_directories=False)
            storage = EvaluationStorage(fp_in)
            results = storage.get_evaluation_data(exp_id, task_type="eval") 
            results = [r for r in results if r["is_best_hyps"]]
            rows_df = pd.DataFrame(results)

            #--------------------------------
            # plot out box plot:   
            #--------------------------------
            x_value = "cust_number"
            x_label = 'Instance Size'
            y_value = 'cummulative_reward'
            y_label = 'Cum. Rewards'
            category = "algorithm"            
            
            original_labels = [key for key, value in manual_legend.items() if key in rows_df.algorithm.unique()]
            new_labels = [value for key, value in manual_legend.items() if key in rows_df.algorithm.unique()]
            
            position += 1
            ax = fig.add_subplot(rows, columns, position)

            # Create a color palette with a consistent order for each unique algorithm
            specific_palette = {old_name: colour for old_name, colour in old_palette.items() if old_name in original_labels}

            if plot_type == "boxplot":
                sns.boxplot(x=x_value, y=y_value, hue=category, data=rows_df, ax=ax, 
                            hue_order=original_labels, palette=specific_palette)
            elif plot_type == "barplot":
                sns.barplot(x=x_value, y=y_value, hue=category, data=rows_df, ax=ax, 
                            hue_order=original_labels, palette=specific_palette)
            else:
                raise ValueError(f'plot_type must be boxplot or barplot')
            
            subplot_solomon_type = solomon_type if solomon_type!= 'O' else 'RC'
            ax.set_title(f'{vrp_variant}, (type-{subplot_solomon_type})', fontsize=25)
            ax.set_xlabel('')
            ax.set_ylabel('')
            ax.tick_params(axis='x', labelsize=16)
            ax.tick_params(axis='y', labelsize=12)

            #--------------------------------
            # Customize the legend. Replace the labels inside the legend
            #--------------------------------
            legend = plt.legend(borderaxespad=0., fontsize=13) #loc='upper left', bbox_to_anchor=(0.06, 0.9), bbox_to_anchor=(1, 1)
            for text, label in zip(legend.get_texts(), new_labels):
                text.set_text(label)
            
    #--------------------------------
    # print out plots in given folder: 
    #--------------------------------
    fig.supxlabel(x_label, fontsize=30)
    fig.supylabel(y_label, fontsize=30)
    plt.tight_layout()
    plt.savefig(f'7_clarks_wright_mdp_evaluation_lineplot.pdf', 
                bbox_inches='tight',
                dpi=300)
    plt.close()


plot_type = "barplot"
mdp_clarks_wright_boxplot(plot_type = plot_type)

### Summary: ALNS evaluation

In [ ]:
clarks_wright_solomon_types = ["O"]

def cw_alns_lineplot_with_ci_summary(budget='all', avg_or_min='obj_avg', y_log=False): 

    sns.set(style='darkgrid')
    fig = plt.figure(figsize=(20, 8))  #width, height
    rows, columns = 2, 3
    position = 0
    
    for solomon_type in clarks_wright_solomon_types: 
        for vrp_variant in vrp_variants: 
            
            #--------------------------------
            # read in corresponding data: 
            #--------------------------------
            exp_id = f'{solomon_type}101200_diffinitcwnoise{vrp_variant}'

            fp_in = FilePaths(parent_dir + exp_7_file_name, exp_id, setup_directories=False)
            storage = EvaluationStorage(fp_in)
            results = storage.get_evaluation_data(exp_id, task_type="alns")  
            results_table = pd.DataFrame(results)
            
            # keep all the budgets or select the specifed one: 
            if budget != 'all': 
                results_table = results_table[(results_table['operator_counter'] == budget)]


            #--------------------------------
            # plot out line plot 
            #--------------------------------
            x_value = "cust_number"
            x_label = 'Instance Size'
            y_value = avg_or_min
            y_label = "Average Objective Value"
            category = 'algorithm'
            
            original_labels = [key for key, value in manual_legend.items() if key in results_table.algorithm.unique()]
            new_labels = [value for key, value in manual_legend.items() if key in results_table.algorithm.unique()]
            
            position += 1
            ax = fig.add_subplot(rows, columns, position)

            # Create a color palette with a consistent order for each unique algorithm
            specific_palette = {old_name: colour for old_name, colour in old_palette.items() if old_name in original_labels}

            sns.lineplot(x=x_value, y=y_value, hue=category, data=results_table, ci=95, 
                         hue_order=original_labels, palette=specific_palette)
            
            subplot_solomon_type = solomon_type if solomon_type!= 'O' else 'RC'
            ax.set_title(f'{vrp_variant}, (type-{subplot_solomon_type})', fontsize=25)
            ax.set_xlabel('')
            ax.set_ylabel('')
            ax.set_xticks([10*x for x in range(2,11)])
            ax.tick_params(axis='x', labelsize=16)
            ax.tick_params(axis='y', labelsize=12)
            
            # Set the y-axis to log scale
            if y_log == True:
                ax.set_yscale("log")
                # ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda y, _: '{:.16g}'.format(y)))

                if vrp_variant == 'TSP':  
                    ax.set_ylim(300, 2200) 
                elif vrp_variant == 'CVRP':  
                    ax.set_ylim(900, 2200) 
                elif vrp_variant == 'HVRP':  
                    ax.set_ylim(900, 2300) 
                elif vrp_variant == 'VRPTW':  
                    ax.set_ylim(900, 4000) 
                elif vrp_variant == 'LRP':  
                    ax.set_ylim(2300, 4500) 


            # Customize the legend. Replace the labels inside the legend
            legend = plt.legend(borderaxespad=0., fontsize=13) #loc='upper left', bbox_to_anchor=(0.06, 0.9), bbox_to_anchor=(1, 1)
            for text, label in zip(legend.get_texts(), new_labels):
                text.set_text(label)

    #--------------------------------
    # print out plots in given folder: 
    #--------------------------------
    fig.supxlabel(x_label, fontsize=30)
    fig.supylabel(y_label, fontsize=30)
    plt.tight_layout()
    plt.savefig(f"7_clarks_wright_evaluation_lineplot.pdf", 
                            bbox_inches='tight',
                            dpi=300)
    plt.close()

cw_alns_lineplot_with_ci_summary(y_log = False)

---
## 8 Put into a single plot

In [ ]:
def create_large_plot_for_all(folder_index, evaluation_type, figure_dir): 

    large_figure = plt.figure(figsize=(20, 20))
    plt.subplots_adjust(wspace=0, hspace=0)  # Remove spacing between subplots

    position = 0
    for vrp_variant in vrp_variants:
        for solomon_type in solomon_types:
            
            ## If we wish to split out only the RC instance: 
            # if solomon_type != "O": 
            #     continue
            
            # Construct the filename for the current figure
            files = os.listdir(f'{figure_dir}/{folder_index}-{evaluation_type}')
            dataset = 'solo' if folder_index != "3" else 'gh'
            matching_files = [f for f in files if (vrp_variant in f) & (dataset+solomon_type in f) & ('min' not in f)]
            
            if matching_files:
                filename = os.path.join(f'{figure_dir}/{folder_index}-{evaluation_type}', matching_files[0])

                # Open the figure using PIL (Pillow)
                img = Image.open(filename)

                # Resize the figure if needed (adjust dimensions as necessary)
                img = img.resize((2400, 1400), Image.ANTIALIAS)

                # Determine the subplot position
                position += 1

                # Add the figure to the large figure at the specified position
                ax = large_figure.add_subplot(len(vrp_variants), len(solomon_types), position)
                ax.imshow(img)
                ax.axis('off')
                
    large_figure.savefig(f"{folder_index}-{evaluation_type}.pdf", 
                         bbox_inches='tight',
                         dpi=300)
    # plt.show()
    plt.close()

In [ ]:
# figure_storage_dir = "/Users/shunee/OneDrive - University of Edinburgh/ATES2022/ATES_Experiments_EJOR2023/figures-2023-10-19"
figure_storage_dir = '/Users/shunee/github/ATES2022/notebooks' 

list_of_experiments = [
                    #    ("1.1", 'mdp-barplot'),
                    #    ('1.2', 'mdp-barplot-with-ci'),
                    #    ('1.3', 'mdp-boxplot'),
                    #    ('2.1', 'alns-lineplot'),
                       ('2.2', 'alns-lineplot-ci'),
                    #    ('3', 'alns-lineplot-large-gh'),
                    #    ('4', 'comput_time_solomon'),
                    #    ('5.1', 'budget-analysis-boxplot'),
                    #    ('5.2', 'budget-analysis-lineplot'), 
                    #    ('6.1', 'mdp-baseline-barplot'),
                    #    ('6.2', 'mdp-baseline-boxplot'),
                    #    ('6.3', 'alns-baseline-lineplot-ci'),
                    #    ('7.2', 'mdp-clarkswright-barplot'),
                    #    ('7.3', 'mdp-clarkswright-boxplot'),
                    #    ('7.4', 'alns-lineplot-clarks-wright'),
                       ]

for (folder_index, evaluation_type) in list_of_experiments: 
    create_large_plot_for_all(folder_index, evaluation_type, figure_storage_dir)

## 9. General comparisons

### Manual Selection

In [ ]:
solomon_type = 'R'
parent_dir = '/Users/shunee/github/All-Experiment-Data/' + exp_2_file_name
all_exp_ids = [f'{solomon_type}101200_diffinitrand'+vrp_variant for vrp_variant in vrp_variants]

vrp_variant = 'LRP'
all_exp_ids = [f'O101200_diffinitrand{vrp_variant}', 
               f'C101200_diffinitrand{vrp_variant}', 
               f'R101200_diffinitrand{vrp_variant}', ]


def get_results_table(storage, exp_id):
    results = storage.get_evaluation_data(exp_id, task_type="alns")
    rows_df = pd.DataFrame(results)
    pivot = pd.pivot_table(rows_df, values='obj_avg', columns=["algorithm"], index=["cust_number"]).reset_index()
    pivot = pd.DataFrame(pivot.to_records()).reset_index(drop=True)
    pivot = pivot[["cust_number", "randommdp", "classicrw", "dqn", "dqnprob", "hybrid", "vrwmdp"]]
    return pivot

all_dfs = []
for exp_id in all_exp_ids:
    fp_in = FilePaths(parent_dir, exp_id, setup_directories=False)
    storage = EvaluationStorage(fp_in)
    exp_df = get_results_table(storage, exp_id)
    details = storage.get_experiment_details(exp_id)
    exp_df['variant'] = details['problem_variant']
    exp_df['inst'] = exp_id[0]
    all_dfs.append(exp_df)

df = pd.concat(all_dfs)

In [ ]:
df

In [ ]:
def percentage_change(col1, col2):
    return ((col2 - col1) / col1) * 100


# WR_dqnprob_dqn = (df["dqnprob"] < df["dqn"]).sum() / len(df) *100
# print(f" DQN-prob over DQN: {WR_dqnprob_dqn:.2f}%")

# WR_hybrid_dqnprob = (df["hybrid"] < df["dqnprob"]).sum() / len(df) *100
# print(f" Hybrid over DQN-prob: {WR_hybrid_dqnprob:.2f}")

# WR_dqn_vrw = (df["dqn"] < df["vrwmdp"]).sum() / len(df) *100
# print(f" DQN over LRW: {WR_dqn_vrw:.2f}")

# WR_hybrid_vrw = (df["hybrid"] < df["vrwmdp"]).sum() / len(df) *100
# print(f" Hybrid over LRW: {WR_hybrid_vrw:.2f}")

# WR_crw_random = (df["classicrw"] < df["randommdp"]).sum() / len(df) *100
# print(f" CRW over RAN: {WR_hybrid_vrw:.2f}")

print()
PC_rand_hybrid = -percentage_change(df["randommdp"], df["hybrid"]).mean()
print(f"Hybrid versus RAN : {PC_rand_hybrid:.2f}%")

PC_rand_dqnprob = -percentage_change(df["randommdp"], df["dqnprob"]).mean()
print(f"DQN-prob versus RAN : {PC_rand_dqnprob:.2f}%")

PC_rand_dqn = -percentage_change(df["randommdp"], df["dqn"]).mean()
print(f"DQN versus RAN : {PC_rand_dqn:.2f}%")

PC_rand_vrw = -percentage_change(df["randommdp"], df["vrwmdp"]).mean()
print(f"LRW versus RAN : {PC_rand_vrw:.2f}%")

PC_rand_crw = -percentage_change(df["randommdp"], df["classicrw"]).mean()
print(f"CRW versus RAN : {PC_rand_crw:.2f}%")

# PC_hybrid_vrw = -percentage_change(df["vrwmdp"], df["hybrid"]).mean()
# print(f"Hybrid versus VRW: {PC_hybrid_vrw:.2f}%")

### For-loop

In [ ]:
def percentage_change(col1, col2):
    return ((col2 - col1) / col1) * 100

def get_results_table(storage, exp_id):
    results = storage.get_evaluation_data(exp_id, task_type="alns")
    rows_df = pd.DataFrame(results)
    pivot = pd.pivot_table(rows_df, values='obj_avg', columns=["algorithm"], index=["cust_number"]).reset_index()
    pivot = pd.DataFrame(pivot.to_records()).reset_index(drop=True)
    pivot = pivot[["cust_number", "randommdp", "classicrw", "dqn", "dqnprob", "hybrid", "vrwmdp"]]
    return pivot

def get_results_table_baseline(storage, exp_id):
    results = storage.get_evaluation_data(exp_id, task_type="alns")
    rows_df = pd.DataFrame(results)
    pivot = pd.pivot_table(rows_df, values='obj_avg', columns=["algorithm"], index=["cust_number"]).reset_index()
    pivot = pd.DataFrame(pivot.to_records()).reset_index(drop=True)
    pivot = pivot[["cust_number", "dqnbaseline", "dqnbaselineprob"]]
    return pivot

def create_pivot_table(all_exp_ids, parent_dir, baseline=False):    
    all_dfs = []
    for exp_id in all_exp_ids:
        fp_in = FilePaths(parent_dir, exp_id, setup_directories=False)
        storage = EvaluationStorage(fp_in)
        if baseline == False: 
            exp_df = get_results_table(storage, exp_id)
        else: 
            exp_df = get_results_table_baseline(storage, exp_id)
        details = storage.get_experiment_details(exp_id)
        exp_df['variant'] = details['problem_variant']
        exp_df['inst'] = exp_id[0]
        all_dfs.append(exp_df)
    return all_dfs


def alns_general_comparisons(input_category):

    #--------------------------------
    # read in corresponding data: 
    #--------------------------------
    if input_category=='O': 
        parent_dir = '/Users/shunee/github/All-Experiment-Data/' + exp_6_file_name_3
        all_exp_ids = [f'{input_category}101200_diffinitrandredone'+vrp_variant for vrp_variant in vrp_variants]
    
    elif input_category=='R' or input_category=='C': 
        parent_dir = '/Users/shunee/github/All-Experiment-Data/' + exp_6_file_name_2
        all_exp_ids = [f'{input_category}101200_diffinitrand'+vrp_variant for vrp_variant in vrp_variants]

    elif input_category in vrp_variants:
        parent_dir = '/Users/shunee/github/All-Experiment-Data/' + exp_6_file_name_2
        all_exp_ids = [f'O101200_diffinitrand{input_category}', 
                       f'C101200_diffinitrand{input_category}', 
                       f'R101200_diffinitrand{input_category}']
    all_dfs = create_pivot_table(all_exp_ids, parent_dir, baseline=False)

    #--------------------------------
    # baseline data:
    #--------------------------------
    parent_dir_base = '/Users/shunee/github/All-Experiment-Data/' + exp_6_file_name_1
    if input_category in solomon_types:
        all_exp_ids_base = [f'{input_category}101200_diffinitrandbaseline{vrp_variant}' for vrp_variant in vrp_variants]
    elif input_category in vrp_variants:
        all_exp_ids_base = [f'O101200_diffinitrandbaseline{input_category}', 
                            f'C101200_diffinitrandbaseline{input_category}', 
                            f'R101200_diffinitrandbaseline{input_category}']
    all_dfs_baseline = create_pivot_table(all_exp_ids_base, parent_dir_base, baseline=True)

    df = pd.concat(all_dfs)
    df_base = pd.concat(all_dfs_baseline)

    #--------------------------------
    # Print out
    #--------------------------------
    
    PC_rand_hybrid = -percentage_change(df["randommdp"], df["hybrid"]).mean()
    print(f"Hybrid versus RAN : {PC_rand_hybrid:.2f}%")

    PC_rand_dqnprob = -percentage_change(df["randommdp"], df["dqnprob"]).mean()
    print(f"DQN-prob versus RAN : {PC_rand_dqnprob:.2f}%")

    PC_rand_dqn = -percentage_change(df["randommdp"], df["dqn"]).mean()
    print(f"DQN versus RAN : {PC_rand_dqn:.2f}%")

    PC_rand_vrw = -percentage_change(df["randommdp"], df["vrwmdp"]).mean()
    print(f"LRW versus RAN : {PC_rand_vrw:.2f}%")

    PC_rand_crw = -percentage_change(df["randommdp"], df["classicrw"]).mean()
    print(f"CRW versus RAN : {PC_rand_crw:.2f}%")

    PC_rand_dqnbase = -percentage_change(df["randommdp"], df_base["dqnbaseline"]).mean()
    print(f"RLB versus RAN : {PC_rand_dqnbase:.2f}%")

    PC_rand_dqnbaseprob = -percentage_change(df["randommdp"], df_base["dqnbaselineprob"]).mean()
    print(f"RLB-M versus RAN : {PC_rand_dqnbaseprob:.2f}%")

In [ ]:
input_category = 'C'
alns_general_comparisons(input_category)

In [ ]:
input_categories = ['R'] + vrp_variants
for input_category in input_categories:
    print(input_category)
    alns_general_comparisons(input_category)
    print()

## 10. Choosing a good plot colour

In [ ]:
# finalised: 
designed_palette = [(56, 167, 208),# GRLOS   - blue
                  (163, 140, 244), # GRLOS-M - purple 
                  (252, 141, 98),  # LRW     - orange
                  (255, 51, 51),   # Hybrid  - red
                  (206, 143, 49),  # CRW     - brown
                  (50, 177, 101),  # RAN     - green
                  (244, 97, 221),  # RLB     - pink 
                  (150, 163, 49),  # RLB-M   - olive 
                  ]

shunee_palette_1 = {'GRLOS':   (56, 167, 208),  # blue
                    'GRLOS-M': (163, 140, 244), # purple 
                    'LRW':     (252, 141, 98),  # orange
                    'Hybrid':  (255, 51, 51),   # red
                    'CRW':     (206, 143, 49),  # brown
                    'RAN':     (50, 177, 101),  # green
                    'RLB':     (244, 97, 221),  # pink 
                    'RLB-M':   (150, 163, 49),  # olive 
                    }
shunee_palette_2 = {'GRLOS':   (0, 114, 178),  # blue
                    'GRLOS-M': (152, 78,  163), # purple 
                    'LRW':     (255, 127, 0),  # orange
                    'Hybrid':  (228, 26,  28),   # red
                    'CRW':     (166, 86,  40),  # brown
                    'RAN':     (0, 158, 115),  # green
                    'RLB':     (247, 129, 191),  # pink 
                    'RLB-M':   (222, 222, 0),  # yellow 
                    }

colors = {
    # 'blue':   [55,  126, 184],  #377eb8 
    # 'orange': [255, 127, 0],    #ff7f00
    # 'green':  [77,  175, 74],   #4daf4a
    # 'pink':   [247, 129, 191],  #f781bf
    # 'brown':  [166, 86,  40],   #a65628
    # 'purple': [152, 78,  163],  #984ea3
    'gray':   [153, 153, 153],  #999999
    # 'red':    [228, 26,  28],   #e41a1c
    'yellow': [222, 222, 0]     #dede00
}  
shu_palette_1 = [tuple(i / 255 for i in c) for c in list(shunee_palette_1.values())]
shu_palette_2 = [tuple(i / 255 for i in c) for c in list(shunee_palette_2.values())]

print(shu_palette_1)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Use Seaborn's color_palette to generate a palette with 8 distinct colors
colors = sns.color_palette("husl", n_colors=8)

# Plot the colors
fig, ax = plt.subplots(figsize=(8, 1))

for i, color in enumerate(colors):
    ax.fill_betweenx(y=[0, 1], x1=i, x2=i+1, color=color)

# Customize the plot
ax.set_xlim(0, 8)
ax.set_ylim(0, 1)
ax.set_xticks([])  # Remove x-axis ticks
ax.set_yticks([])  # Remove y-axis ticks
ax.set_title('8 Distinct Colors')

plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Use Seaborn's color_palette to generate a palette with 8 distinct colors
colors = sns.color_palette("husl", n_colors=8)

# Update the 1st and 5th colors for better distinction
first_color = (255, 51, 51)
first_color = [x/255 for x in first_color]
colors[0] = first_color
colors[4] = sns.color_palette("Set2", n_colors=2)[1]

# Plot the updated colors
fig, ax = plt.subplots(figsize=(8, 1))

for i, color in enumerate(colors):
    ax.fill_betweenx(y=[0, 1], x1=i, x2=i+1, color=color)

# Customize the plot
ax.set_xlim(0, 8)
ax.set_ylim(0, 1)
ax.set_xticks([])  # Remove x-axis ticks
ax.set_yticks([])  # Remove y-axis ticks
ax.set_title('Updated 8 Distinct Colors')

plt.show()

print(colors[4])

In [ ]:
# V1
import seaborn as sns
import matplotlib.pyplot as plt

colors = sns.color_palette("husl", n_colors=8)

# Plot the colors
fig, ax = plt.subplots(figsize=(8, 1))

for i, color in enumerate(shu_palette_1):
    ax.fill_betweenx(y=[0, 1], x1=i, x2=i+1, color=color)

# Customize the plot
ax.set_xlim(0, 8)
ax.set_ylim(0, 1)
ax.set_xticks([])  # Remove x-axis ticks
ax.set_yticks([])  # Remove y-axis ticks
ax.set_title('8 Distinct Colors')

plt.show()

In [ ]:
# V2
import seaborn as sns
import matplotlib.pyplot as plt

colors = sns.color_palette("husl", n_colors=8)

# Plot the colors
fig, ax = plt.subplots(figsize=(8, 1))

for i, color in enumerate(shu_palette_2):
    ax.fill_betweenx(y=[0, 1], x1=i, x2=i+1, color=color)

# Customize the plot
ax.set_xlim(0, 8)
ax.set_ylim(0, 1)
ax.set_xticks([])  # Remove x-axis ticks
ax.set_yticks([])  # Remove y-axis ticks
ax.set_title('8 Distinct Colors')

plt.show()